# Two Lattices

If we split our lattice into two lattices, 2n+1 neighborly and not ... what do we get?


In [1]:
import sys, io
import math
import numpy as np
import pandas as pd
from scipy.optimize import nnls
from fractions import Fraction
from sympy import factorint
from itertools import product, combinations
from typing import List, Optional, Tuple
import random
import matplotlib.pyplot as plt



https://oeis.org/A003586


In [2]:
def C(n):
    """ Compute next value in simplified Collatz sequence.
    """
    if n & 1 == 0:
        return n//2
    else:
        return (3*n + 1)//2
#
def L_C(n):
    """ Compute binary label-string for a given Collatz number
    """
    if n == 1:
        return "1"

    S = ""
    while n != 1:
        if n & 1 == 0:
            n = n//2
            S = S + "1"
        else:
            n = (3*n + 1)//2
            S = S + "0"
    return S
#
def collatzVector(collatzNumber):
    chain = [collatzNumber]
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
        chain.append(collatzNumber)
    chain.append(chain[-2])
    return np.array(chain)
    
def collatzPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#

In [3]:
def Ay_L(L):
    """ Generate A matrix and y vector from label string
    """
    rank = len(L) + 2    
    A = np.zeros((rank,rank))
    y = np.zeros((rank))
    for row in range(rank-3):
        if L[row] == "1":
            a_val = -1.0
            y_val = 0.0
        else:
            a_val = -3.0
            y_val = 1.0
        A[row][row] = a_val
        A[row][row+1] = 2.0
        y[row] = y_val
    #
    # Last 3 rows are always the same
    row = rank - 3
    A[row][row] = -1
    A[row][row+1] = 2
    y[row] = 0
    row = rank - 2
    A[row][row] = -3
    A[row][row+1] = 2
    y[row] = 1
    row = rank - 1
    A[row][row] = -1
    A[row][row-2] = 1
    y[row] = 0
    
    return A, y
#

In [4]:
Ay_L("00111")

(array([[-3.,  2.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -3.,  2.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  2.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  2.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  2.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -3.,  2.],
        [ 0.,  0.,  0.,  0.,  1.,  0., -1.]]),
 array([1., 1., 0., 0., 0., 1., 0.]))

In [5]:
def solve_Ay_L(L):
    """ Solve for the x vector given the label-string
    """
    A, y = Ay_L(L)
    return A, np.linalg.solve(A, y), y
#

In [6]:
solve_Ay_L("00111")

(array([[-3.,  2.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -3.,  2.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  2.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  2.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  2.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -3.,  2.],
        [ 0.,  0.,  0.,  0.,  1.,  0., -1.]]),
 array([3., 5., 8., 4., 2., 1., 2.]),
 array([1., 1., 0., 0., 0., 1., 0.]))

In [7]:
def x0_L(L):
    """ Get the x[0] value given a label-string
    """
    A, x, y = solve_Ay_L(L)
    return round(x[0])  # clean up mantisa garbage
#

In [8]:
x0_L("00111")

3

In [9]:
def countZeros(label):
    zero_count = 0
    for bit in label:
        if bit == "0":
            zero_count += 1
    return zero_count
#

def Z(L):
    """ Indexes of zeros in label string
    """
    for i in range(len(L)):
        if L[i] == "0":
            yield i
#

In [10]:
list(Z("00111"))

[0, 1]

In [11]:
def a_b_c_S(L):
    """ Get the (power-of-two, power-of-three, zero-sum-accumulator) tuple for a node given its label
    """
    a = len(L)
    b = 0
    for bit in L:
        if bit == "0":
            b += 1
    ZZ = [(j,i) for j, i in enumerate(Z(L))]
    c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in ZZ)
    S = [zz[1] for zz in ZZ]
    return (a,b,c,S)

In [12]:
a_b_c_S("01")

(2, 1, 1, [0])

In [13]:
a_b_c_S("00111")

(5, 2, 5, [0, 1])

In [14]:
def val_a_b_c(a_b_c):
    """ Get the value for a node given the tuple (power-of-two, power-of-three, zero-sum-accumulator)
    """
    a, b, c = a_b_c
    f = Fraction( ((2**a) - c), (3**b) )
    return (f.numerator, f.denominator)
#
def val_a_b_c_S(a_b_c_S):
    a_b_c = (a_b_c_S[0], a_b_c_S[1], a_b_c_S[2])
    return val_a_b_c(a_b_c)
#

In [15]:
val_a_b_c((5, 2, 5))

(3, 1)

In [16]:
def val_L(L):
    """ Get the value for a node given the label string
    """
    return val_a_b_c_S(a_b_c_S(L))
#

In [17]:
val_L("00111")

(3, 1)

In [18]:
N_i = ((0,0), [])

In [19]:
def mr_TupItemValue(a_b, a_0):
    """ Value of (a, b) tuple with a_0 added to b so we are not dealing with float limitations
    """
    a,b = a_b
    return (2**a)*(3**(a_0 + b))
#
def mrTupValue(mr_tup):
    """ Compute the value of the given mrTup

        Returns value as a cannonical (numerator, denominator) pair
    """
    # Multiplying the numerator by 3 ** the generation keeps us in integer land
    a_0 = mr_tup[0][0]
    total = mr_TupItemValue(mr_tup[0], a_0)
    for a_b in mr_tup[1]:
        total -= mr_TupItemValue(a_b, a_0)
    frac = Fraction(total, 3**a_0)
    return (frac.numerator, frac.denominator)
#

In [20]:
mrTupValue(N_i)

(1, 1)

In [21]:
def F_0(mr_tup):
    u_tup, v_list = mr_tup
    a,b = u_tup
    u_tup_ = (a+1, b-1)
    if len(v_list) > 0:
        b_ = v_list[-1][1] - 1
    else:
        b_ = b - 1
    v_list = [(vt[0], vt[1]) for vt in v_list] + [(a, b_ )]
    return (u_tup_, v_list)
#

In [22]:
# Terse version for math conversion
def F_0(mr_tup):
    return ( (mr_tup[0][0]+1, mr_tup[0][1]-1), mr_tup[1] + [(mr_tup[0][0], mr_tup[0][1]-1)] )
#

In [23]:
def F_1(mr_tup):
    u_tup, v_list = mr_tup
    a,b = u_tup
    u_tup = (a+1, b)
    return (u_tup, v_list)
#

In [24]:
def F_1(mr_tup):
    return ((mr_tup[0][0]+1, mr_tup[0][1]), mr_tup[1])
#

In [25]:
mrTupValue(F_1(N_i))

(2, 1)

In [26]:
F_0(N_i)

((1, -1), [(0, -1)])

In [27]:
mrTupValue(F_0(N_i))

(1, 3)

In [28]:
mrTupValue(F_0(F_0(N_i)))

(-1, 9)

In [29]:
F_0(F_0(N_i))
# 4/9 - 1/9 -2/3

((2, -2), [(0, -1), (1, -2)])

In [30]:
mrTupValue(F_0(F_0(N_i)))

(-1, 9)

In [31]:
mrTupValue(F_0(F_0(F_0(N_i))))

(-11, 27)

In [32]:
mrTupValue(F_0(F_0(F_1(N_i))))

(-2, 9)

In [33]:
def F_rev(mr_tup):
    """ Reverse of either F0 or F1, uses contents of L list to determine COA
    """
    u_tup, v_list = mr_tup
    a,b = u_tup
    a_ = a-1
    b_ = b
    if len(v_list) > 0:
        if v_list[-1][0] == a_:
            b_ = b+1
            if len(v_list) > 1:
                v_list =  [(vt[0], vt[1]) for vt in v_list[0:-1]]
            else:
                v_list = []
    return ((a_, b_), v_list)
#
    

In [34]:
F_rev(F_rev(F_0(F_0(N_i))))

((0, 0), [])

In [35]:
F_rev(F_rev(F_1(F_1(N_i))))

((0, 0), [])

In [36]:
"""
G_0, G_1 work from the other side of the label prepending to the label

These parallel the Collatz Sequence
"""
def G_0(mr_tup):
    return ( (mr_tup[0][0]+1, mr_tup[0][1]-1), [(0,-1)] + [(c+1, d-1) for (c, d) in mr_tup[1]] )
#
def G_1(mr_tup):
    return ((mr_tup[0][0]+1, mr_tup[0][1]), [(c+1, d) for (c, d) in mr_tup[1]])
#



In [37]:
def G_rev(mr_tup):
    """ Reverse of either G_0 or G_1, uses contents of L list to determine COA
    """
    u_tup, v_list = mr_tup
    a,b = u_tup
    a_ = a-1
    b_ = b
    if len(v_list) > 0:
        if v_list[0][0] == 0 and v_list[0][1] == -1:
            # Then we are undoing a G_0 ...
            b_ = b+1
            if len(v_list) == 1:
                v_list = []
            else:
                v_list = [(c-1, d+1) for (c, d) in mr_tup[1][1:]]
        else:
            v_list = [(c-1, d) for (c, d) in mr_tup[1]]
    return ((a_, b_), v_list)
#


In [38]:
collatzVector(13)

array([13, 20, 10,  5,  8,  4,  2,  1,  2])

In [39]:
def mrTupFromPath(label):
    """ Create an mrTup given a path (label)
    """
    mr_tup = N_i
    for bit in label:
        if bit == "1":
            mr_tup = F_1(mr_tup)
        else:
            mr_tup = F_0(mr_tup)
    return mr_tup
#
def mrTupFromValue(n):
    label = collatzPath(n)
    return mrTupFromPath(label)
#
def mrTupToLaTex(T):
    a, b = T[0]
    s = "\\frac{2^{%d}}{3^{%d}}"%(a, -b)
    L = T[1]
    if len(L) > 0:
        s = s + " - ( "
        plus = "  "
        for c_d in L:
            c, d = c_d
            t = "\\frac{2^{%d}}{3^{%d}}"%(c, -d)
            s = s + plus + t
            plus = " + "
        s = s + " )"
    return "$ " + s + " $"
#

In [40]:
def rationalCollatzPath(fraction_tup, max_length=100):
    """ Apply Collatz rules to a rational to see if it has a path to 1 and is in the lattice 
    """
    numerator_0, denominator_0 = fraction_tup
    rational_collatz_path = [(numerator_0, denominator_0)]
    numerator, denominator = fraction_tup
    for i in range(max_length):
        if numerator == 1 and denominator == 1:
            break
        if (abs(numerator) & 1) == 1:
            numerator = 3*numerator + denominator
        denominator = 2*denominator
        f = Fraction(numerator, denominator)
        numerator = f.numerator
        denominator = f.denominator
        # Check for a cycle
        if (numerator, denominator) in rational_collatz_path:
            rational_collatz_path.append((numerator, denominator))
            break
        rational_collatz_path.append((numerator, denominator))

    return rational_collatz_path

In [41]:
def gen_generation(a):
    """ Generate a (label, a_b_c_tuple, numerator_denominator_pair) tuple for each
        lattice node in a generation

        Note: uses what we have learned regarding the mapping of the label string
        to create a nodes L subtractand list -- does not require applying F_0, F_1

        Fastest way to generate a generation's nodes
    """
    seqs = product('10', repeat=a)
    for bits in seqs:
        label = ''.join(bits)
        zeros = [i for i, b in enumerate(bits) if b == '0']
        b = len(zeros)
        # compute c = sum_{j=0}^{k} 3^{k-j} * 2^{i_j - 1}
        c = sum((3 ** (b - j - 1)) * (2 ** (i)) for j, i in enumerate(zeros))
        f = Fraction(2**a - c, 3**b)
        yield (label, (a,b,c), (f.numerator, f.denominator))
#

In [42]:
def mr2Nplus_1(T):
    B = len(T[1])  
    L = [(0, -1)]

    # Keep initial zeros
    idx = 0
    for idx, val in enumerate(T[1]):
        if T[1][idx][0] != idx:
            break
        L.append( (T[1][idx][0] + 1, T[1][idx][1]-1) )
    # Remove the first tuple where (a, -a) is true
    match = False
    for i in range(idx, B, 1):
        if (not match) and (T[1][i][0] == -T[1][i][1]):
            match = True
        else:
            L.append( (T[1][i][0]+1, T[1][i][1]) )
    if not match:
        return None
    return ( (T[0][0] + 1, T[0][1]), L)
#


In [43]:
def mrTupToLaTex(T):
    a, b = T[0]
    s = "\\frac{2^{%d}}{3^{%d}}"%(a, -b)
    L = T[1]
    if len(L) > 0:
        s = s + " - ( "
        plus = "  "
        for c_d in L:
            c, d = c_d
            t = "\\frac{2^{%d}}{3^{%d}}"%(c, -d)
            s = s + plus + t
            plus = " + "
        s = s + " )"
    return "$ " + s + " $"
#

In [44]:
def shortParity(k, n):
    parity_bits = []
    for i in range(k):
        if n == 1:  
            return None
        n_ = C(n)
        if n_ < n:
            parity_bits.append("0")
        else:
            parity_bits.append("1")
        n = n_
    return "".join(parity_bits)
# 
def verifyShortParityMod(k, test_count=2**10):
    mod_base = 2**k
            
    # generate 2^a slots for modulus mappings
    mapping = [""]*(mod_base)
    for i in range(mod_base, test_count+mod_base): # made sure starting label isn't shorter than test target length
        mod = i % mod_base
        parity_str = shortParity(k, i)
        if parity_str is not None:  # A number was given that has a shorter path than k
            if len(mapping[mod]) > 0:
                if parity_str != mapping[mod]:
                    print("Non-unique label map!")
            else:
                # First time
                mapping[mod] = parity_str
#

In [45]:
# No output == good
verifyShortParityMod(3)

In [46]:
def verifyUniqueMapping(prefix_length, test_count=2**10):
    mod_base = 2**prefix_length
            
    # generate 2^a slots for modulus mappings
    mapping = [""]*(mod_base)
    for i in range(mod_base, test_count+mod_base): # made sure starting label isn't shorter than test target length
        mod = i % mod_base
        label = collatzPath(i)
        if len(mapping[mod]) > 0:
            if label[0:prefix_length] != mapping[mod]:
                print("Non-unique label map!")
        else:
            # First time
            mapping[mod] = label[0:prefix_length]
#
verifyUniqueMapping(3)

In [47]:
# No output => No errors.
verifyUniqueMapping(6)

In [48]:
solve_Ay_L(collatzPath(20))

(array([[-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  2.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -3.,  2.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  2.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  2.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  2.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -3.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0., -1.]]),
 array([20., 10.,  5.,  8.,  4.,  2.,  1.,  2.]),
 array([0., 0., 1., 0., 0., 0., 1., 0.]))

$$
\begin{bmatrix}
\mathbf{-1} & \mathbf{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\
0 & \mathbf{-1} & \mathbf{2} & 0 & 0 & 0 & 0 & 0 & 0 \\\\
0 & 0 & \mathbf{-3} & \mathbf{2} & 0 & 0 & 0 & 0 & 0 \\\\
0 & 0 & 0 & -3 & 2 & 0 & 0 & 0 & 0 \\\\
0 & 0 & 0 & 0 & -1 & 2 & 0 & 0 & 0 \\\\
0 & 0 & 0 & 0 & 0 & -1 & 2 & 0 & 0 \\\\
0 & 0 & 0 & 0 & 0 & 0 & -1 & 2 & 0 \\\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & -3 & 2 \\\\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & -1
\end{bmatrix}
\begin{bmatrix}
12 \\\\
6 \\\\
3 \\\\
5 \\\\
8 \\\\
4 \\\\
2 \\\\
1 \\\\
2
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{0} \\\\
\mathbf{0} \\\\
\mathbf{1} \\\\
1 \\\\
0 \\\\
0 \\\\
0 \\\\
1 \\\\
0
\end{bmatrix}
$$

In [49]:
solve_Ay_L(collatzPath(3*12 + 4))

(array([[-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  2.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -3.,  2.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  2.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0., -1.,  2.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  2.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -3.,  2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0., -1.]]),
 array([40., 20., 10.,  5.,  8.,  4.,  2.,  1.,  2.]),
 array([0., 0., 0., 1., 0., 0., 0., 1., 0.]))

$$
\begin{bmatrix}
\mathbf{-1} & \mathbf{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\\\
0 & \mathbf{-1} & \mathbf{2} & 0 & 0 & 0 & 0 & 0 & 0 \\\\
0 & 0 & \mathbf{-1} & \mathbf{2} & 0 & 0 & 0 & 0 & 0 \\\\
0 & 0 & 0 & -3 & 2 & 0 & 0 & 0 & 0 \\\\
0 & 0 & 0 & 0 & -1 & 2 & 0 & 0 & 0 \\\\
0 & 0 & 0 & 0 & 0 & -1 & 2 & 0 & 0 \\\\
0 & 0 & 0 & 0 & 0 & 0 & -1 & 2 & 0 \\\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & -3 & 2 \\\\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & -1
\end{bmatrix}
\begin{bmatrix}
40 \\\\
20 \\\\
10 \\\\
5 \\\\
8 \\\\
4 \\\\
2 \\\\
1 \\\\
2
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{0} \\\\
\mathbf{0} \\\\
\mathbf{0} \\\\
1 \\\\
0 \\\\
0 \\\\
0 \\\\
1 \\\\
0
\end{bmatrix}
$$

## The Full List of 3-bit Mappings that Always work 
Only the $\mapsto $ "111" were required for the proof

In [50]:
def T_010_to_T_111(k):  # 1(mod 8)
    return 9*k + 7
#
def T_010_to_T_110(k):
    return 3*k + 1
#
def T_010_to_T_011(k):
    return 3*k + 2
#
def T_101_to_T_111(k):
    return 3*k + 2
#
def T_001_to_T_111(k): # 3(mod 8)
    return 9*k + 5
#
def T_001_to_T_101(k):
    return 3*k + 1
#
def T_110_to_T_111(k):
    return 3*k + 4
#
def T_110_to_T_011(k):
    return k + 1
#
def T_011_to_T_111(k): # 5(mod 8)
    return 3*k + 1
#
def T_011_to_T_110(k):
    return k - 1
#
def T_011_to_T_100(k):
    # Not affine at 3 bits ...
    if (k%24) == 21:
        return (k//3) -1
    else:
        return k+2
#
def T_100_to_T_111(k):
    return 9*k + 10
#
def T_100_to_T_110(k):
    return 3*k + 2
#
def T_100_to_T_011(k):
    return 3*k + 3
#
def T_000_to_T_111(k):  # 7 (mod 8)
    return 27*k + 19
#
def T_000_to_T_110(k):
    return 9*k + 5
#
def T_000_to_T_011(k):
    return 9*k + 6
#
def T_000_to_T_100(k):
    return 3*k + 1
#

def checkMod8Formula(mod_8, tranform_func, prefix):
    for i in range(1000):
        k = 8*(i+2) + mod_8
        label = collatzPath(k)
        label_ = prefix + label[len(prefix):]
        val_ = mrTupValue(mrTupFromPath(label_))
        if val_[1] != 1:
            print(f"Label substitution FAILED {k}({label}) -> ({label_}) has noninteger {val_}")
            return False
        k_ = tranform_func(k) 
        if k_ != val_[0]:
            print(f"{tranform_func.__name__} FAILED {k} should -> {val_} but function gave {k_}")
            return False
    return True
#
[
    checkMod8Formula(1, T_010_to_T_111, "111"),
    checkMod8Formula(1, T_010_to_T_110, "110"),
    checkMod8Formula(1, T_010_to_T_011, "011"),
    
    checkMod8Formula(2, T_101_to_T_111, "111"),
    checkMod8Formula(3, T_001_to_T_111, "111"),
    
    checkMod8Formula(3, T_001_to_T_101, "101"),
    
    checkMod8Formula(4, T_110_to_T_111, "111"),
    checkMod8Formula(4, T_110_to_T_011, "011"),
    
    checkMod8Formula(5, T_011_to_T_111, "111"),
    checkMod8Formula(5, T_011_to_T_110, "110"),
    
    checkMod8Formula(6, T_100_to_T_111, "111"),
    checkMod8Formula(6, T_100_to_T_110, "110"),
    checkMod8Formula(6, T_100_to_T_011, "011"),
    
    checkMod8Formula(7, T_000_to_T_111, "111"),
    checkMod8Formula(7, T_000_to_T_110, "110"),
    checkMod8Formula(7, T_000_to_T_011, "011"),
    checkMod8Formula(7, T_000_to_T_100, "100")
]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

### Symbolic Determinants of the non "111*" Matrices

A symbolic look at the numerator $Det(A_{x_0})$ matrices for the solution approach $x = \Large{\frac{Det(A_{x_0})}{Det(A)}}$

Where $ζ$ is $z-1$ when label bit is "0" or $z$ when label bit is "1"

In [51]:
from sympy import Matrix, MatrixSymbol, symbols
A = Matrix([[symbols('y0'), 2, 0], [symbols('y1'), symbols('b'), 2], [symbols('ζ'), 0, symbols('c')]])
det_A_0 = A.det()
det_A_0

b*c*y0 - 2*c*y1 + 4*ζ

In [52]:
A = Matrix([[symbols('y0'), 2, 0, 0], [symbols('y1'), symbols('b'), 2, 0], [symbols('y2'), 0, symbols('c'), 2], [symbols('ζ'), 0, 0, symbols('d')]])
det_A_0 = A.det()
det_A_0

b*c*d*y0 - 2*c*d*y1 + 4*d*y2 - 8*ζ

In [53]:
A = Matrix([
    [symbols('y0'), 2, 0, 0, 0], 
    [symbols('y1'), symbols('b'), 2, 0, 0], 
    [symbols('y2'), 0, symbols('c'), 2, 0], 
    [symbols('y3'), 0, 0, symbols('d'), 2],
    [symbols('ζ'), 0, 0, 0, symbols('e')]])
det_A_0 = A.det()
det_A_0

b*c*d*e*y0 - 2*c*d*e*y1 + 4*d*e*y2 - 8*e*y3 + 16*ζ

E.g. the pattern:
$$
terms =[ (-1)^0(y_0, b, c, d, e),  (-1)^1(2, y_1, c, d, e),  (-1)^2(2, 2, y_2, d, e),  (-1)^3(2, 2, 2, y_3, e), (-1)^4(2, 2, 2, 2, ζ) ]
$$
$$
    det(A_{x_0}) = \sum_{products}\prod_{terms}term
$$


- The determinant of $A_{111}$ is always  $-1^{rank}$
- The $x_0$ determinant of $A_{111,x_0}$ is always $2^{rank -1}$

So the $x_0$ equation for the "111" equation is always:

$x_0' = -1^{rank} \cdot 2^{rank -1}z$

- The determinant of $A$ is always $-1^{rank} \cdot 3^{zerocount}$
- The $x_0$ determinant of $A_{x_0}$ above is more complicated because of the ones that can appear in y.

# Generalize Isomorphic Transform

I have implemented the generalized form of the isomorphic transform for any $\pmod{2^a}$

This was not needed for the proof.

## The prefix finding function

Gives us the correct prefix which is really the secret code for affine function transform paramters


In [54]:
def computeNextPrefixBit(a, p2, label, mod):
    # We cannot choose two label bits, so we generate an exemplar
    example = p2 + mod 
    label = collatzPath(example) 
    return a+1, 2**(a+1), label[0:a]
    
def computePrefix(n):
    mod = n % 4
    label = ["11", "01", "10", "00"][mod]
    if n > 3:
        a = 3
        p2 = 2**(a)
        while  p2 < n:
            a, p2, label = computeNextPrefixBit(a, p2, label, n % p2)
    return label

## Mapping Matrix Sketch

This function generates matrices like the ones we need for the solution to give us some insight

In [55]:
def generateMappingMatrices(prefix):
    """ NOTIONAL:
        Generate the matrices and vectors (except for the y "z-term")
        used to discover the T_P -> T_111* mapping
    """
    rank = len(prefix)
    A = np.zeros((rank, rank))
    y = np.zeros((rank))
    A_111 = np.zeros((rank, rank))
    y_111 = np.zeros((rank))
    
    for i in range(rank):
        A_111[i][i] = -1
    for i in range(rank-1):
        A[i][i+1] = 2
        A_111[i][i+1] = 2
    # Except for the "z-term" the y_111 vector is always 0

    for i in range(rank):
        if prefix[i] == "1":
            A[i][i] = -1
            y[i] = 0
        else:
            A[i][i] = -3
            y[i] = 1
    print((A, y))
    print(np.linalg.det(A))
    print((A_111, y_111))
    print(np.linalg.det(A_111))


In [56]:
generateMappingMatrices("000")

(array([[-3.,  2.,  0.],
       [ 0., -3.,  2.],
       [ 0.,  0., -3.]]), array([1., 1., 1.]))
-27.0
(array([[-1.,  2.,  0.],
       [ 0., -1.,  2.],
       [ 0.,  0., -1.]]), array([0., 0., 0.]))
-1.0


## The per-prefix mapping function

For a given prefix, creates the correct affine parameters and applies them to the parameter x.


In [57]:
def mapByPrefix(prefix, x):
    """
    Given the prefix of a number (computed from its (mod 2**len(prefix))
    Compute its mapped value in the 111* partition of the lattice
    """
    a = len(prefix)
    b = countZeros(prefix)
    accum = 0
    sgn = -1**a
    for i in range(a-1):
        sgn = sgn * -1
        P = [2]*(a) # we do not do the zz term
        if prefix[i] == "0":
            P[i] = 1
        else:
            continue  # y = 0, so product will be 0
        for j in range(i+1, a, 1):
            if prefix[j] == "0":
                P[j] = -3
            else:
                P[j] = -1
        #print((sgn, P))
        accum += (sgn * np.prod(P))

    if prefix[-1] == "0":
        #print((-sgn, [2]*(a-1)))
        accum -= (sgn * (2**(a-1)))
    #print(b, accum)
    return int((3**b) * x + accum) # get rid of np.int before return
#

In [58]:
[
    40 == mapByPrefix("110", 12), # 4 (mod 8) 3k+4 = 40
    56 == mapByPrefix("101", 18), # 2 (mod 8) 3k+2 = 56
    424 == mapByPrefix("000", 15) # 7 (mod 8) 27k+19 = 405 + 19 = 424
]

[True, True, True]

## map any number up to "111"

This function puts the above together to take an integer and return the affine mapped result

In [59]:
def mapNumberUp(n):
    """ return the 0 (mod 2^a) number that is the transform of a number from a different part of the lattice
    """
    prefix = computePrefix(n)
    n_ = mapByPrefix(prefix, n)
    return n_
#

### Exercise of pretending a number is not in the lattice

Note: we quickly reach the limits of even Python's large integer capabilities

In [60]:
def applyMod8Affine(n):
    def NOOP(n):
        return n
    #
    LU = {
        0: NOOP, # prefix "111",
        1: T_010_to_T_111, # prefix "010",
        2: T_101_to_T_111, # prefix "101"
        3: T_001_to_T_111, # prefix "001"
        4: T_110_to_T_111, # prefix "110"
        5: T_011_to_T_111, # prefix "011"
        6: T_100_to_T_111, # prefix "100"
        7: T_000_to_T_111  # prefix "000"
    }
    mod_8 = n % 8
    return (LU[mod_8])(n)
#
def divide2n(n):
    while (n & 1) == 0:
        n >>= 1
    return n
#
applyMod8Affine(11), divide2n(4)

(104, 1)

In [61]:
# Lets pretend 357 is not in the lattice
# As expected, we wind up at 1 by applying the affine transforms and dividing by 2
N = 357

In [62]:
n = N
for i in range(8):
    if n == 1:
        break
    n_ = applyMod8Affine(n)
    n__ = divide2n(n_)
    print((n, n_, n__))
    n = n__

(357, 1072, 67)
(67, 608, 19)
(19, 176, 11)
(11, 104, 13)
(13, 40, 5)
(5, 16, 1)


In [63]:
# Generate generation-sized random bit tester of the algorithm
def random_bit_test_run_mod8(generations_a):
    failure_count = 0
    iteration_stats = {}
    def collect_iteration(b):
        if b not in iteration_stats:
            iteration_stats[b] = 0
        iteration_stats[b] = iteration_stats[b] + 1
    #
    
    # generate bits at generation_a and rotate through them to test.
    bits = list(np.random.randint(2, size=generations_a, dtype=np.uint8))
    for i in range(2**(generations_a)-1):
        bits = bits[1:] + [bits[0]]
        val = 0
        for bit in bits:
            val = (val << 1) | bit
        n = val
        if n == 0:
            continue
        for i in range(4*generations_a):
            if n == 1:
                break
            n_ = applyMod8Affine(int(n))
            n__ = divide2n(n_)
            n = n__
        if n != 1:
            # Only print the ones that fail to reduce
            print((n, n_, n__))
            failure_count = failure_count + 1
        collect_iteration(i)
    return((failure_count, iteration_stats))
#

In [64]:
# As we increase the generation size, the maximum number of steps required seems to approach the number of generation bits
# We see as many as 17 steps when we limit ourselves to integers of 8 bits, but we see a max of 21 steps for integers of 20 bits.

# Does a larger modulus decay faster?

Yes, so long as it is from an odd power of 2 and $2^a < n$


In [65]:
def computePrefixForModClass(power_of_2, n):
    mod = n % 4
    label = ["11", "01", "10", "00"][mod]
    if power_of_2 > 2:
        a = 3
        p2 = 2**(a)
        while a <= power_of_2:
            a, p2, label = computeNextPrefixBit(a, p2, label, n % p2)
    return label
#

In [66]:
# Regenerate our mod_8 label table ... looks correct
for i in range(8):
    n = i + 16
    print( (n%8, computePrefixForModClass(3, n)) )

(0, '111')
(1, '010')
(2, '101')
(3, '001')
(4, '110')
(5, '011')
(6, '100')
(7, '000')


In [67]:
computePrefixForModClass(3, 73)

'010'

In [254]:
def generationLabels(a):
    """
    Yields the 2^a label strings of the labels/prefixes for a given generation
    """
    if a == 0:
        return ""
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        yield label
#
def shortenedGenerationLabels(a):
    """
    Yields the 2^a label strings of the labels/prefixes for a given generation
    """
    if a == 0:
        return ""
    seqs = product('10', repeat=(a))
    for bit_tup in seqs:
        label = "".join(bit_tup)
        while label[-2:] == "01":
            label = label[0:-2]
        yield label
#


In [69]:
def affineFunctionParamsFromPrefix(prefix):
    """
    Return the affine parameters A, B for n' = A*n + B

    (Logic extracted from earlier mapByPrefix function)
    """
    a = len(prefix)
    b = countZeros(prefix)
    accum = 0
    sgn = -1**(a)
    for i in range(a-1):
        sgn = sgn * -1
        P = [2]*(a) # we do not do the zz term
        if prefix[i] == "0":
            P[i] = 1
        else:
            continue  # y = 0, so product will be 0
        for j in range(i+1, a, 1):
            if prefix[j] == "0":
                P[j] = -3
            else:
                P[j] = -1
        #print((sgn, P))
        accum += (sgn * np.prod(P))

    if prefix[-1] == "0":
        #print((-sgn, [2]*(a-1)))
        accum -= (sgn * (2**(a-1)))
    #print(b, accum)
    return(3**b, int(accum)) # get rid of np.int before returning
#

In [70]:
def affineFunctionFromModulus(power_of_2, mod):
    """
    Generates the correct An+B function for the given modulus and base.
    """
    mod_base = 2**power_of_2
    prefix = computePrefixForModClass(power_of_2, mod+mod_base) # Avoid zero, one conditions with added mod_base
    A, B = affineFunctionParamsFromPrefix(prefix)
    # print(f"{mod}(mod {mod_base}): A={A}, B={B}")
    def an_affine_func(n):
        return A*n + B 
    #
    return an_affine_func
#    

In [71]:
# Check new formula generator against old (mod 8) tester
for modulus in range(0, 8, 1):
    f = affineFunctionFromModulus(3, modulus)
    checkMod8Formula(modulus, f, "111")
#
    

In [72]:
def checkFormula(modulus, tranform_func, prefix):
    a = len(prefix)
    modulus_base = 2**a
    for i in range(1000):
        k = (modulus_base*(i+2)) + modulus
        label = collatzPath(k)
        label_ = prefix + label[len(prefix):]
        val_ = mrTupValue(mrTupFromPath(label_))
        if val_[1] != 1:
            return False
        k_ = tranform_func(k) 
        if k_ != val_[0]:
            return False
    return True
#


In [73]:
# Swapping new general checking function for checkMod8Formula
# Should give same result
for modulus in range(0, 8, 1):
    f = affineFunctionFromModulus(3, modulus)
    checkFormula(modulus, f, "111")
#

In [74]:
# No output is good news
for modulus in range(1, 16, 1):
    f = affineFunctionFromModulus(4, modulus)
    checkFormula(modulus, f, "1111")
#

In [75]:
def random_bit_test_run(power_2, generations_a):
    """
    Applies our generated affine functions to random selection of bits from the given generation's representation
    and then rotates those bits for interesting, consistent examples.

    Tests for convergence, which works for all odd powers of two, but not even powers of two.
    """
    modulus_base = 2**power_2
    affine_dict = {}
    for i in range(modulus_base):
        affine_dict[i] = affineFunctionFromModulus(power_2, i)
    #
    def applyAffine(n):
        mod = n % modulus_base
        return (affine_dict[mod])(n)
    #
        
    failure_count = 0
    iteration_stats = {}
    def collect_iteration(b):
        if b not in iteration_stats:
            iteration_stats[b] = 0
        iteration_stats[b] = iteration_stats[b] + 1
    #
    
    # generate bits at generation_a and rotate through them to test.
    bits = list(np.random.randint(2, size=generations_a, dtype=np.uint8))
    for i in range(2**(generations_a)-1):
        bits = bits[1:] + [bits[0]]
        val = 0
        for bit in bits:
            val = (val << 1) | bit
        n = val
        if n == 0:
            continue
        for i in range(4*generations_a):
            if n == 1:
                break
            n_ = applyAffine(int(n))
            n__ = divide2n(n_)
            n = n__
        if n != 1:
            # Only print the ones that fail to reduce
            print((n, n_, n__))
            failure_count = failure_count + 1
        collect_iteration(i)
    return((failure_count, iteration_stats))
#

In [76]:
random_bit_test_run(5, 12)

(0, {1: 1025, 2: 342, 11: 682, 13: 341, 3: 1023, 4: 341, 14: 341})

# Odd generations converge to 1, Even generations diverge
- Odd generations have positive $B$ values in the affine transform and converge.
    - These operations "look like" aggregated 3n+1 operations
- Even generations have negative $B$ values in the affine transform and diverge.
    - The operations do not "look like" aggregated 3n+1 operations.

? What does "looks like aggregated 3n+1 operations" really mean?

? The negative B values make it more likely that the divide by 2 operations after promotion will land one back in the lowest part of the lattice, the positive B values keep 

# Max Convergence Function

We know how to implement maximum convergence for any Collatz integer.

This allows us to move any number to 1 in as few steps as possible


In [77]:
def maxOddPowerOf2(n):
    p2 = math.floor(math.log(n, 2.0))
    if p2 & 1 != 1:
        p2 -=1
    if p2 < 3:
        p2 = 3
    return p2
#
def maxConverge(n):
    trace = []  # Collect A, B tup and power of 2 reduced for each step
    while n != 1:
        p2 = maxOddPowerOf2(n)
        prefix = computePrefixForModClass(p2, n)
        A, B = affineFunctionParamsFromPrefix(prefix)
        n = A*n + B
        j = 0
        while n & 1 == 0:
            n >>=1
            j+=1
        trace.append(( len(collatzPath(n)), p2, (A, B), j, n, n%3))
    return trace
#

In [78]:
collatzPath(357)

'01110011100110010110111'

In [79]:
maxConverge(357)

[(14, 7, (27, 89), 9, 19, 1),
 (10, 3, (9, 5), 4, 11, 2),
 (7, 3, (9, 5), 3, 13, 1),
 (4, 3, (3, 1), 3, 5, 2),
 (0, 3, (3, 1), 4, 1, 1)]

### Accelerated Convergence $2^{40} - 1$ : 343 -> 16 steps
- The shortcut Collatz sequence for $2^{40} - 1$ takes 343 steps to converge to 1
- The rapid MaxZ sequence takes just 16 steps ... over $20\times$ faster

# For paper we just need (mod 8) to converge every time

So lets get back to that ...


In [80]:
r = 1
next_odd_mod_stats = [0]*8
p2_dict = {}
def collect_p2(j):
    if j not in p2_dict:
        p2_dict[j] = 0
    p2_dict[j] += 1
#
for i in range(1024):
    n = 8*i + r + 512
    prefix = computePrefixForModClass(3, n)
    A, B = affineFunctionParamsFromPrefix(prefix)
    n_ = A*n + B
    j = 0
    while n_ & 1 == 0:
        n_ >>=1
        j+=1
    collect_p2(j)
    next_odd_mod_stats[n_ % 8] += 1
next_odd_mod_stats, p2_dict

([0, 256, 0, 255, 0, 256, 0, 257],
 {4: 256,
  3: 512,
  5: 128,
  6: 64,
  9: 8,
  7: 32,
  8: 16,
  11: 2,
  10: 4,
  12: 1,
  16: 1})

In [81]:
r = 3
next_odd_mod_stats = [0]*8
p2_dict = {}
def collect_p2(j):
    if j not in p2_dict:
        p2_dict[j] = 0
    p2_dict[j] += 1
#
for i in range(1024):
    n = 8*i + r + 512
    prefix = computePrefixForModClass(3, n)
    A, B = affineFunctionParamsFromPrefix(prefix)
    n_ = A*n + B
    j = 0
    while n_ & 1 == 0:
        n_ >>=1
        j+=1
    collect_p2(j)
    next_odd_mod_stats[n_ % 8] += 1
next_odd_mod_stats, p2_dict

([0, 255, 0, 256, 0, 257, 0, 256],
 {5: 128,
  3: 512,
  4: 256,
  6: 64,
  7: 32,
  9: 8,
  8: 16,
  10: 4,
  12: 1,
  11: 2,
  13: 1})

In [82]:
r = 5
next_odd_mod_stats = [0]*8
p2_dict = {}
def collect_p2(j):
    if j not in p2_dict:
        p2_dict[j] = 0
    p2_dict[j] += 1
#
for i in range(1024):
    n = 8*i + r + 512
    prefix = computePrefixForModClass(3, n)
    A, B = affineFunctionParamsFromPrefix(prefix)
    n_ = A*n + B
    j = 0
    while n_ & 1 == 0:
        n_ >>=1
        j+=1
    collect_p2(j)
    next_odd_mod_stats[n_ % 8] += 1
next_odd_mod_stats, p2_dict

([0, 257, 0, 256, 0, 256, 0, 255],
 {4: 256,
  3: 512,
  6: 64,
  5: 128,
  8: 16,
  7: 32,
  9: 8,
  12: 1,
  10: 4,
  11: 2,
  14: 1})

In [83]:
r = 7
next_odd_mod_stats = [0]*8
p2_dict = {}
def collect_p2(j):
    if j not in p2_dict:
        p2_dict[j] = 0
    p2_dict[j] += 1
#
for i in range(1024):
    n = 8*i + r + 512
    prefix = computePrefixForModClass(3, n)
    A, B = affineFunctionParamsFromPrefix(prefix)
    n_ = A*n + B
    j = 0
    while n_ & 1 == 0:
        n_ >>=1
        j+=1
    collect_p2(j)
    next_odd_mod_stats[n_ % 8] += 1
next_odd_mod_stats, p2_dict

([0, 255, 0, 256, 0, 256, 0, 257],
 {4: 256,
  3: 512,
  7: 32,
  5: 128,
  6: 64,
  9: 8,
  8: 16,
  10: 4,
  11: 2,
  14: 1,
  12: 1})

# Odd -> Even -> Odd bipartite graph on Odd

We see that the Odd (N) -> Even (N') -> Odd operation (N'') gives an even distributiion of N'' across the 4 possible modulus classes.  Each gets 25% no matter what we start with.  The intermediate Even (N') can wind up being any even number $2^3\cdot2^d\cdot\prod_{i}f_i \text{ where } f_i \text{ are factors }> 2$ and it is the unknown factors that ultimately choose what the next modulus for $N''$ will be.

Reminder the functions are:

```
def T_010_to_T_111(k):  # 1(mod 8)
    return 9*k + 7
#
def T_001_to_T_111(k): # 3(mod 8)
    return 9*k + 5
#
def T_011_to_T_111(k): # 5(mod 8)
    return 3*k + 1
#
def T_000_to_T_111(k):  # 7(mod 8)
    return 27*k + 19
#
```


# Convergence of the affine Collatz Shift Function Z

Given:
$$
	W(n)=
	\begin{cases}
	9n + 7, & \text{if } n \equiv 1\pmod{8},\\
	9n + 5, & \text{if } n \equiv 3\pmod{8},\\
	3n + 1, & \text{if } n \equiv 5\pmod{8},\\
	27n + 19, & \text{if } n \equiv 7\pmod{8}.
	\end{cases}
$$

We can define a Collatz-like sequence function that is mappable to the lattice:

$$
	Z(n)=
	\begin{cases}
		\dfrac{n}{2}, & \text{if } n \text{ is even},\\
		W(n), & \text{if } n \text{ is odd}.
	\end{cases}
$$



In [84]:
AB = [(9, 7), (9, 5), (3, 1), (27, 19)]
mappings = []
for r in [1, 3, 5, 7]:
    for i in range(256):
        idx_odd = (r-1)//2
        A, B = AB[idx_odd]
        n = 8*i + r
        n_ = A*n + B
        n__ = n_
        while n__ & 1 == 0:
            n__ >>=1
        mappings.append(((A, B), n, n%8, n__%8))  # n_ % 8 is always 0, so do not track it
#print(mappings)
mappings_counts = {}
for tup in mappings:
    if tup[0] not in mappings_counts:
        mappings_counts[tup[0]] = {}
    mapping = (tup[2], tup[3])
    if mapping not in mappings_counts[tup[0]]:
        mappings_counts[tup[0]][mapping] = 0
    mappings_counts[tup[0]][mapping] += 1
mappings_counts

{(9, 7): {(1, 1): 64, (1, 3): 64, (1, 5): 65, (1, 7): 63},
 (9, 5): {(3, 1): 64, (3, 5): 64, (3, 3): 64, (3, 7): 64},
 (3, 1): {(5, 1): 65, (5, 5): 64, (5, 3): 64, (5, 7): 63},
 (27, 19): {(7, 5): 63, (7, 3): 64, (7, 1): 65, (7, 7): 64}}

Every $1,3,5,7 \pmod{8}$ class has $\approx 1/4$ chance of going to any other $1,3,5,7 \pmod{8}$ class

So for a long series we have the convergence ratio:

$$
\Large{\frac{3^1\cdot3^2\cdot3^2\cdot3^3}{2^3\cdot2^3\cdot2^3\cdot2^3\cdot2^d}}
$$
Where $d$ accounts for the additional divide by twos from half of all operations, and these "extra twos" (beyond the built-in 3 we get due to $0\pmod{8}$ are usually larger than just a single 2 -- and the farthe left in the lattice we go, the more extra twos we get.


# This is the crux of it:

The step options are very evenly distributed.  We have created a situation where we are absolutely guaranteed that we will get this ratio ... and better.

We only need one more two on the bottom to have convergence every time.  

In [85]:
# When d is 0:
(3*9*9*27)/(8*8*8*8)

1.601806640625

In [86]:
def countTwos(n):
    i = 0
    while (n & 1) == 0:
        n >>= 1
        i+=1
    return i
#
def distributionAverage(D):
    accum = 0
    count = 0
    R = {0: 0.0, 2: 0.0, 4: 0.0, 6: 0.0}
    for mod in D:
        for key in D[mod]:
            count += D[mod][key]
            accum += (key * D[mod][key])
        avg = accum / count
        R[mod] = avg
    return R
#
def distributionOfTwos(a):
    """
    For the given generation, count how many twos can divided out of a given number:
    """
    D = {}
    just_3 = []
    def collectCount(val, count):
        if count == 3:
            reduce = val
            while (reduce & 1 ) == 0:
                reduce >>= 1
            just_3.append((val, reduce%8))
        mod = val % 8
        if mod in [0,2,4,6]:
            if mod not in D:
                D[mod] = {}
            if count not in D[mod]:
                D[mod][count] = 0
            D[mod][count] = D[mod][count] + 1
    #
    for label in generationLabels(a):
        val = mrTupValue(mrTupFromPath(label))
        if val[1] == 1:
            collectCount(val[0], countTwos(val[0]))
    return distributionAverage(D), D, just_3
#

In [87]:
distributionOfTwos(3)

({0: 3.0, 2: 2.0, 4: 0.0, 6: 0.0}, {0: {3: 1}, 2: {1: 1}}, [(8, 1)])

In [88]:
# 0(mod 8)
#   - Three Twos: 1
#   - More : 1
distributionOfTwos(5)

({0: 4.0, 2: 2.5, 4: 0.0, 6: 0.0}, {0: {5: 1, 3: 1}, 2: {1: 2}}, [(8, 1)])

In [89]:
# 0(mod 8)
#   - Three Twos: 2
#   - More : 2
distributionOfTwos(7)

({0: 4.5, 2: 2.875, 4: 4.0, 6: 0.0},
 {0: {7: 1, 5: 1, 3: 2}, 4: {2: 1}, 2: {1: 3}},
 [(40, 5), (8, 1)])

In [90]:
# 0(mod 8)
#   - Three Twos: 3
#   - More : 5
distributionOfTwos(9)

({0: 4.875, 2: 3.357142857142857, 4: 4.3, 6: 0.0},
 {0: {9: 1, 7: 1, 5: 2, 4: 1, 3: 3}, 4: {2: 2}, 2: {1: 4}},
 [(168, 5), (40, 5), (8, 1)])

In [91]:
# 0(mod 8)
#   - Three Twos: 4
#   - More : 10
distributionOfTwos(11)

({0: 5.357142857142857, 2: 3.7083333333333335, 4: 4.611111111111111, 6: 3.5},
 {0: {11: 1, 9: 1, 7: 2, 6: 1, 5: 3, 4: 2, 3: 4},
  4: {2: 4},
  2: {1: 6},
  6: {1: 2}},
 [(680, 5), (168, 5), (40, 5), (8, 1)])

In [92]:
# 0(mod 8)
#   - Three Twos: 8 (big jump)
#   - More : 18
distributionOfTwos(13)

({0: 5.5, 2: 3.760869565217391, 4: 4.527777777777778, 6: 3.54},
 {0: {13: 1, 11: 1, 9: 2, 8: 1, 7: 3, 6: 2, 5: 4, 4: 4, 3: 8},
  4: {2: 10},
  2: {1: 10},
  6: {1: 4}},
 [(2728, 5), (680, 5), (168, 5), (40, 5), (904, 1), (8, 1), (280, 3), (88, 3)])

In [93]:
# 0(mod 8)
#   - Three Twos: 14 (big jump)
#   - More : 32
distributionOfTwos(15)

({0: 5.54, 2: 3.9047619047619047, 4: 4.641791044776119, 6: 3.741573033707865},
 {0: {15: 1, 13: 1, 11: 2, 10: 1, 9: 3, 8: 2, 7: 4, 6: 4, 5: 8, 4: 10, 3: 14},
  4: {2: 17},
  2: {1: 17},
  6: {1: 5}},
 [(10920, 5),
  (2728, 5),
  (680, 5),
  (168, 5),
  (3624, 5),
  (40, 5),
  (1128, 5),
  (360, 5),
  (904, 1),
  (8, 1),
  (280, 3),
  (88, 3),
  (1208, 7),
  (120, 7)])

In [94]:
takes_a_while ="""
distributionOfTwos(25)

({0: 5.987341772151899,
  2: 4.189917127071824,
  4: 5.00259965337955,
  6: 4.011082138200782},
 {0: {25: 1,
   23: 1,
   21: 2,
   20: 1,
   19: 3,
   18: 2,
   17: 4,
   16: 4,
   15: 8,
   14: 10,
   13: 14,
   12: 17,
   11: 22,
   10: 28,
   9: 40,
   8: 51,
   7: 69,
   6: 90,
   5: 121,
   4: 161,
   3: 220},
  4: {2: 285},
  2: {1: 294},
  6: {1: 86}})
"""

So we have:

| Generation | $\textbf{max(2 Factor)}$ | $\langle\textbf{2 Factor}\rangle$ | $\lvert\textbf{2 Factor}\rvert$=3 | $\lvert\textbf{2 Factor}\rvert>3$ |
| ---: | ---: |  ---: |  ---: |  ---: |
| 3  | 3  | 3.0 | 1 | 0 |
| 5   | 5   | 4.0 | 1 | 1 |
| 7   | 7   | 4.5 | 2 | 1 |
| 9   | 9   | 4.875 | 3 | 5 |
| 11  | 11  | 5.35 | 4 | 10 |
| 13  | 13  | 5.5 | 8 | 18 |
| 15  | 15  | 5.54 | 14 | 32 |




# ^^^ Count_of_Two >> 3 for 0(mod8)
We can see that for 0(mod 8), an integer only having the minimal 3 factors of two is less common than an integer having more factors of two than this, often many more factors.  The "availability of extra twos" increases as we go to the right in the lattice.

Which is why $Z(n)$ converges so quickly in practice

# Can a series exist that only uses the $27n+19$ transform?

No finite number can start such a series, because every time we add a step the modular constraints on n_0 increase exponentially.

- 


## The Z(n) Sequence converges 



In [95]:
# TODO: put modulus tables, etc here.

| k(mod8) | 3k+2(mod8) | New Class (mod8) | 
| :--- | :--- | :--- |
| Even ($0, 2, 4, 6$) | $2, 8, 14, 20 \equiv 2, 0, 6, 4$ | $\mathbf{1, 3, 5, 7}$ (Odd) |
| Odd ($1, 3, 5, 7$) | $5, 11, 17, 23 \equiv 5, 3, 1, 7$ | $\mathbf{1, 3, 5, 7}$ (Odd) |


| k(mod8) | k+4(mod8) | New Class (mod8)| 
| :--- | :--- | :--- |
| 1 | $1+4 = 5$ | $\mathbf{5 \pmod{8}}$ |
| 3 | $3+4 = 7$ | $7 \pmod{8}$ |
| 5 | $5+4 = 9 \equiv 1$ | $1 \pmod{8}$ |
| 7 | $7+4 = 11 \equiv 3$ | $3 \pmod{8}$ |


In [96]:
9841 % 8

1

## ^^^ NOT 7.

We can only get a sequence length $\ell$ of $7\pmod{8}$ numbers from a number $n_0$ that is $(2^{3\ell} -1)\pmod{2^{3\ell}}$

$511 =  (2^{3\ell} -1) \text{ for } \ell = 3$ so we got a run of three $7\pmod{8}$ numbers before getting something else.

Aparently any time one tries to pin a Collatz sequence to a modulus group you get this effect of ever-increasing modulus constraints on the intial number of a sequence.  This is related to Lagarias' number density findings among other Collatz related topics on this subject.

In [97]:
def triple_bad(n_0):
    n_1 = 27*n_0 + 19
    while n_1 & 1 == 0:
        n_1 >>= 1
    n_2 = 27*n_1 + 19
    while n_2 & 1 == 0:
        n_2 >>= 1
    n_3 = 27*n_2 + 19
    while n_3 & 1 == 0:
        n_3 >>= 1
    return (n_3, n_3 % 8)

In [98]:
triple_bad(511), triple_bad(511+512), triple_bad(511+(2*512)), triple_bad(511+(3*512))

((9841, 1), (39365, 5), (7381, 5), (78731, 3))

In [99]:
f = Fraction( -(8*(2**5)) - 11, 27 )
f.numerator, f.denominator

(-89, 9)

In [100]:
mrTupValue(F_0(F_0(F_0(mrTupFromPath("11111")))))

(-352, 27)

In [101]:
"""
The following functions implement a rapidly converging twist on the Collatz Conjecture. 
When an initial number is chosen that forces a large number of diverging steps, 
it does not seem to matter how large the initial number of diverging steps is
and the algorithm recovers begins converging after two steps, recovers to approximately the 
initial log(n, 2) within 5 steps and then rapidly converges to one.

Will this always recover in 5 steps?  Can a bound on worse case converging times be
found for this form of the problem?
"""
def collatzPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#
def computeNextPrefixBit(a, p2, label, mod):
    # We cannot choose two label bits, so we generate an exemplar
    example = p2 + mod 
    label = collatzPath(example) 
    return a+1, 2**(a+1), label[0:a]
    
def computePrefixForModClass(power_of_2, n):
    mod = n % 4
    label = ["11", "01", "10", "00"][mod]
    if power_of_2 > 2:
        a = 3
        p2 = 2**(a)
        while a <= power_of_2:
            a, p2, label = computeNextPrefixBit(a, p2, label, n % p2)
    return label
#

def maxOddPowerOf2(n):
    p2 = math.floor(math.log(n, 2.0))
    if p2 & 1 != 1:
        p2 -=1
    if p2 < 3:
        p2 = 3
    return p2
#
def affineFunctionParamsFromPrefix(prefix):
    """
    Return the affine parameters A, B for n' = A*n + B

    (Logic extracted from earlier mapByPrefix function)
    """
    a = len(prefix)
    b = countZeros(prefix)
    accum = 0
    sgn = -1**(a)
    for i in range(a-1):
        sgn = sgn * -1
        P = [2]*(a) # we do not do the zz term
        if prefix[i] == "0":
            P[i] = 1
        else:
            continue  # y = 0, so product will be 0
        for j in range(i+1, a, 1):
            if prefix[j] == "0":
                P[j] = -3
            else:
                P[j] = -1
        #print((sgn, P))
        # Trying to avoid an overflow condition ...
        if sgn == 1:
            accum = accum + np.prod(P)
        else:
            accum = accum - np.prod(P)

    if prefix[-1] == "0":
        #print((-sgn, [2]*(a-1)))
        accum -= (sgn * (2**(a-1)))
    #print(b, accum)
    return(3**b, int(accum)) # get rid of np.int before returning
#
def acceleratedConvergeOdd(n):
    p2 = maxOddPowerOf2(n)  # Get the largest odd power of 2 less than n
    """
    Identify and apply the maximum affine mapping to
    move the odd parameter to the 0(mod 2^{p2}) portion of the lattice
    """
    prefix = computePrefixForModClass(p2, n)
    A, B = affineFunctionParamsFromPrefix(prefix)
    n_ = A*n + B
    return n_
#
def acceleratedConvergeEven(n):
    """
    Remove all powers of two from n to get the next odd number
    """
    n_ = n
    while n_ & 1 == 0:
        n_ >>=1
    return n_
#

# Force m divergent initial steps
m = 11
n = 2**(3*m) - 1
step_count = 0
while n != 1:
    n = acceleratedConvergeOdd(n)
    n = acceleratedConvergeEven(n)
    step_count += 1
#
print(step_count)


18


In [102]:
# Force a m divergent initial steps
m = 11
n = 2**(3*m) - 1
step_count = 0
while n != 1:
    n = acceleratedConvergeOdd(n)
    n = acceleratedConvergeEven(n)
    step_count += 1
#
print(step_count)


18


In [103]:
def collatzPath(collatzNumber):
    path = []
    while collatzNumber != 1:
        if (collatzNumber & 1) == 0:
            collatzNumber = collatzNumber // 2
            path.append("1")
        else:
            collatzNumber = (3 * collatzNumber + 1) // 2
            path.append("0")
    return "".join(path)
#
def computeNextPrefixBit(a, p2, label, mod):
    # We cannot choose two label bits, so we generate an exemplar
    example = p2 + mod 
    label = collatzPath(example) 
    return a+1, 2**(a+1), label[0:a]
    
def computePrefixForModClass(power_of_2, n):
    mod = n % 4
    label = ["11", "01", "10", "00"][mod]
    if power_of_2 > 2:
        a = 3
        p2 = 2**(a)
        while a <= power_of_2:
            a, p2, label = computeNextPrefixBit(a, p2, label, n % p2)
    return label
#


In [104]:
def prefixForModClass(power_of_2, n):
    mod = n % 4
    prefix = ["11", "01", "10", "00"][mod]
    if power_of_2 > 2:
        a = 3
        while a <= power_of_2:
            p2 = 2**(a)
            mod = n % p2
            exemplar = p2 + mod
            bits = []
            while exemplar != 1:
                if (exemplar & 1) == 0:
                    bits.append("1")
                    exemplar = exemplar // 2
                else:
                    bits.append("0")
                    exemplar = (3 * exemplar + 1) // 2
            next_bit = bits[a-1]
            prefix = prefix + next_bit
            a = a+1
    return prefix
#


In [105]:
prefixForModClass(4, 27), computePrefixForModClass(4,27)

('0010', '0010')

In [106]:
def bottom_val(a):
    """
    OEIS A002450

    Not really the "bottom value", more like "top of bottom 1/4" for even generations and "top of bottom half" for odd generations

    Has 1:1 mapping via applyMod8Affine to 2^n
    """
    if a < 2:
        return None
    if a & 1 == 1:
        return 2*bottom_val(a-1)
    j = a//2 
    val = 0
    for i in range(j):
        val <<=2
        val |= 1
    return val
#
    

In [107]:
bottom_val(2), bottom_val(3), bottom_val(4), bottom_val(5), bottom_val(6), bottom_val(7), bottom_val(8), bottom_val(9), bottom_val(10), bottom_val(11)

(1, 2, 5, 10, 21, 42, 85, 170, 341, 682)

In [108]:
for i in range(2, 14):
    n = bottom_val(i)
    label = collatzPath(n)
    print((n, label, applyMod8Affine(n)))

(1, '', 16)
(2, '1', 8)
(5, '0111', 16)
(10, '10111', 32)
(21, '011111', 64)
(42, '1011111', 128)
(85, '01111111', 256)
(170, '101111111', 512)
(341, '0111111111', 1024)
(682, '10111111111', 2048)
(1365, '011111111111', 4096)
(2730, '1011111111111', 8192)


In [109]:
for i in range(2, 9):
    n = ((3**i) - (2**i))
    print((n, collatzPath(n)))

(5, '0111')
(19, '00110010110111')
(65, '0101011100010110111')
(211, '001100011011100110010110111')
(665, '01001101001100011011100110010110111')
(2059, '001011011100111100011110111')
(6305, '0101001000111110111000011011100110010110111')


# Can we always choose a larger modulus to overcome 27n+19?

In [110]:
n = 27
p2 = 3 # normal choice: largest odd power of 2 less than 27
prefix = computePrefixForModClass(p2, 27)
print(prefix)
A, B = affineFunctionParamsFromPrefix(prefix)
print((A, B))

001
(9, 5)


In [111]:
9*27 + 5

248

In [112]:
248//8

31

In [113]:
# 31 is > 27 ... so
prefix = computePrefixForModClass(p2+2, 27)
print(prefix)
A, B = affineFunctionParamsFromPrefix(prefix)
print((A, B))

00100
(81, 85)


In [114]:
81*27 + 95

2282

In [115]:
2282//2

1141

In [116]:
# 1141 is larger than 27
math.log(1141, 2)

10.156083076288683

In [117]:
prefix = computePrefixForModClass(p2+2+12, 27)
print(prefix)
A, B = affineFunctionParamsFromPrefix(prefix)
print((A, B))

00100000101001000
(1594323, 2828479)


In [118]:
1594323*27 + 2828479

45875200

In [119]:
from sympy import factorint

In [120]:
factorint(45875200)

{2: 18, 5: 2, 7: 1}

In [121]:
45875200//(2**18)

175

In [122]:
# 175 is larger than 27
math.log(175, 2)

7.45121111183233

In [123]:
prefix = computePrefixForModClass(p2+2+12+8, 27)
print(prefix)
A, B = affineFunctionParamsFromPrefix(prefix)
print((A, B))

0010000010100100010000101
(387420489, 746827085)


In [124]:
387420489*27 + 746827085

11207180288

In [125]:
factorint(11207180288)

{2: 26, 167: 1}

In [126]:
11207180288//(2**26)

167

In [127]:
math.log(167, 2)

7.383704292474053

In [128]:
prefix = computePrefixForModClass(p2+2+12+8+8, 27)
print(prefix)
A, B = affineFunctionParamsFromPrefix(prefix)
print((A, B))

001000001010010001000010110001001
(94143178827, 198323306519)


In [129]:
94143178827*27 + 198323306519

2740189134848

In [130]:
factorint(2740189134848)

{2: 33, 11: 1, 29: 1}

In [131]:
2740189134848//(2**33)

319

In [132]:
math.log(319, 2)

8.31741261376487

In [133]:
prefix = computePrefixForModClass(p2+2+12+8+8+10, 27)
print(prefix)
A, B = affineFunctionParamsFromPrefix(prefix)
print((A, B))

0010000010100100010000101100010010000001100
(617673396283947, 1363605088882039)


In [134]:
617673396283947*27 + 1363605088882039

18040786788548608

In [135]:
factorint(18040786788548608)

{2: 43, 7: 1, 293: 1}

In [136]:
18040786788548608//(2**43)

2051

In [137]:
math.log(2051, 2)

11.002111776479852

In [138]:
prefix = computePrefixForModClass(p2+2+12+8+8+10+12, 27)
print(prefix)
A, B = affineFunctionParamsFromPrefix(prefix)
print((A, B))

0010000010100100010000101100010010000001100001110101011
(150094635296999121, 342958083294627829)


In [139]:
150094635296999121*27 + 342958083294627829

4395513236313604096

In [140]:
factorint(4395513236313604096)

{2: 56, 61: 1}

In [141]:
4395513236313604096//(2**56)

61

In [142]:
math.log(61)

4.110873864173311

In [143]:
"""
prefix = computePrefixForModClass(p2+2+12+8+8+10+12+6, 27)
print(prefix)
A, B = affineFunctionParamsFromPrefix(prefix)
print((A, B))
"""
"""
/tmp/ipykernel_666828/1125787258.py:72: RuntimeWarning: overflow encountered in scalar add
  accum = accum + np.prod(P)
"""

'\n/tmp/ipykernel_666828/1125787258.py:72: RuntimeWarning: overflow encountered in scalar add\n  accum = accum + np.prod(P)\n'

# Compare numerator and denominator of maxConverge steps

We always get diagonals for both the numerator and denominator of the solution for the affine function.
There must be a way to show that there are always more twos that will come out of the numerator than
the denominator.  We only need one extra two per step ... looks like we always get at least 3.

## So how do we show:

### $n >> d$
### in
### $Det_{numerator}\pmod{2^n}$ versus $Det_{denominator}\pmod{2^d}$


In [144]:
maxConverge(63)

[(63, 5, (243, 211), 5, 485, 2),
 (56, 7, (27, 89), 7, 103, 1),
 (51, 5, (81, 73), 5, 263, 2),
 (44, 7, (81, 73), 7, 167, 2),
 (37, 7, (243, 251), 7, 319, 1),
 (29, 7, (729, 665), 8, 911, 2),
 (20, 9, (243, 323), 9, 433, 1),
 (11, 7, (27, 85), 9, 23, 2),
 (4, 3, (27, 19), 7, 5, 2),
 (0, 3, (3, 1), 4, 1, 1)]

In [145]:
computePrefixForModClass(5, 63)

'00000'

In [146]:
def affineFunctionParamsFromPrefix(prefix):
    """
    Return the affine parameters A, B for n' = A*n + B

    (Logic extracted from earlier mapByPrefix function)
    """
    a = len(prefix)
    b = countZeros(prefix)
    accum = 0
    sgn = -1**(a)
    for i in range(a-1):
        sgn = sgn * -1
        P = [2]*(a) 
        if prefix[i] == "0":
            P[i] = 1
        else:
            continue  # y = 0, so product will be 0
        for j in range(i+1, a, 1):
            if prefix[j] == "0":
                P[j] = -3
            else:
                P[j] = -1
        #print((sgn, P))
        accum += (sgn * np.prod(P))

    if prefix[-1] == "0":
        #print((-sgn, [2]*(a-1)))
        accum -= (sgn * (2**(a-1)))
    #print(b, accum)
    return(3**b, int(accum)) # get rid of np.int before returning
#

In [147]:
def efficient_binary_arrangements(num_zeros, num_ones):
    """
    Generates all unique arrangements of a string with a given number 
    of black and white beads efficiently using combinations.
    """
    total_length = num_zeros + num_ones
    # Define bead types
    black = '0'
    white = '1'

    # Get all combinations of indices where black beads will be placed
    # This returns an iterator of tuples of indices
    black_indices_combinations = combinations(range(total_length), num_zeros)
    
    unique_arrangements = []
    
    # Iterate directly over the unique combinations
    for black_indices in black_indices_combinations:
        # Create a list representation of the current arrangement
        arrangement_list = [white] * total_length
        for i in black_indices:
            arrangement_list[i] = black

        yield "".join(arrangement_list)
#

# Example usage for 3 black and 2 white beads (total 5 beads)
black_beads = 2
white_beads = 3
arrangements = efficient_binary_arrangements(black_beads, white_beads)

print(f"Unique arrangements for {black_beads} black and {white_beads} white beads:")
for arr in arrangements:
    print(arr)

Unique arrangements for 2 black and 3 white beads:
00111
01011
01101
01110
10011
10101
10110
11001
11010
11100


In [148]:
by_zeros = {1:{}, 2:{}, 3:{}}

for i in range(2, 20, 1):
    k = i - 1
    if k > 4:
        k = 4
    for j in range(1,k,1):
        for head in efficient_binary_arrangements(j, i-j):
            label = head + "111"
            T = mrTupFromPath(label)
            val = mrTupValue(T)
            if val[1] == 1:
                by_zeros[j][label] = (val, val[0]%(3**j), T)
by_zeros

{1: {'011111': ((21, 1), 0, ((6, -1), [(0, -1)])),
  '110111': ((20, 1), 2, ((6, -1), [(2, -1)])),
  '1011111': ((42, 1), 0, ((7, -1), [(1, -1)])),
  '1110111': ((40, 1), 1, ((7, -1), [(3, -1)])),
  '01111111': ((85, 1), 1, ((8, -1), [(0, -1)])),
  '11011111': ((84, 1), 0, ((8, -1), [(2, -1)])),
  '11110111': ((80, 1), 2, ((8, -1), [(4, -1)])),
  '101111111': ((170, 1), 2, ((9, -1), [(1, -1)])),
  '111011111': ((168, 1), 0, ((9, -1), [(3, -1)])),
  '111110111': ((160, 1), 1, ((9, -1), [(5, -1)])),
  '0111111111': ((341, 1), 2, ((10, -1), [(0, -1)])),
  '1101111111': ((340, 1), 1, ((10, -1), [(2, -1)])),
  '1111011111': ((336, 1), 0, ((10, -1), [(4, -1)])),
  '1111110111': ((320, 1), 2, ((10, -1), [(6, -1)])),
  '10111111111': ((682, 1), 1, ((11, -1), [(1, -1)])),
  '11101111111': ((680, 1), 2, ((11, -1), [(3, -1)])),
  '11111011111': ((672, 1), 0, ((11, -1), [(5, -1)])),
  '11111110111': ((640, 1), 1, ((11, -1), [(7, -1)])),
  '011111111111': ((1365, 1), 0, ((12, -1), [(0, -1)])),
  '1

In [149]:
# The single-zero numbers directly:
b = 1
for a in range(6, 12):
    print(f"a={a}:--------------")
    if (a & 1) == 0:
        i_0 = 0
    else:
        i_0 = 1
    for i in range(i_0, a-3, 2):
        n = (2**a  - 2**i)//3
        print(f"   i={i}: {n}" )


a=6:--------------
   i=0: 21
   i=2: 20
a=7:--------------
   i=1: 42
   i=3: 40
a=8:--------------
   i=0: 85
   i=2: 84
   i=4: 80
a=9:--------------
   i=1: 170
   i=3: 168
   i=5: 160
a=10:--------------
   i=0: 341
   i=2: 340
   i=4: 336
   i=6: 320
a=11:--------------
   i=1: 682
   i=3: 680
   i=5: 672
   i=7: 640


In [150]:
# First category of double-zero numbers directly:
# == Fixed zero just before terminating "111"
# == i_0 parity choice flips compared to single-zero direct generation
z2_cat1 = []

b = 1
for a in range(6, 12):
    print(f"a={a}:--------------")
    if (a & 1) == 0:
        i_0 = 1
    else:
        i_0 = 0
    for i in range(i_0, a-3, 2):
        n = (2**a  - 3*(2**i) - 2**(a-4))//9
        z2_cat1.append(n)
        print(f"   i={i}: {n}" )

a=6:--------------
   i=1: 6
a=7:--------------
   i=0: 13
   i=2: 12
a=8:--------------
   i=1: 26
   i=3: 24
a=9:--------------
   i=0: 53
   i=2: 52
   i=4: 48
a=10:--------------
   i=1: 106
   i=3: 104
   i=5: 96
a=11:--------------
   i=0: 213
   i=2: 212
   i=4: 208
   i=6: 192


In [151]:
def direct_0(a):
    """
    This function directly generates all integers of the given generation $a$
    that have one zero in their label
    """
    collatzNums = []
    if (a & 1) == 0:
        i_0 = 0
    else:
        i_0 = 1
    for c0 in range(i_0, a-3, 2):
        n = (2**a  - 2**c0)//3
        collatzNums.append(n)
    return collatzNums
#

In [152]:
direct_0(10)

[341, 340, 336, 320]

In [153]:
direct_0(11)

[682, 680, 672, 640]

In [154]:
(10-2)//2, (11-2)//2

(4, 4)

In [155]:
(12-2)//2, (13-2)//2

(5, 5)

# direct\_0 has (a-2)//2 solutions per generation

In [156]:
def fit_total_soln_odd_0():    
    x = []
    y = []
    for i in range(5, 40, 2):
        x.append(i)
        y.append(len(direct_0(i)))
    return x, y
fit_total_soln_odd_0()

([5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])

In [157]:
def directTup_00(a):
    """
    This function directly generates all mrTyps of the given generation $a$
    that have two zeros in their label
    """
    def swapParity(parity_):
        return parity_ ^ 1
    #
    
    b = 2
    c1 = a-4
    delta_c01 = [1, 2]
    delta_c0 = [4, 2]
    parity = 0
    while c1 >= 1:
        c0 = c1 - delta_c01[parity]
        while c0 >= 0:
            n = (2**a - 3*(2**c0) - (2**c1))//9
            yield ((a, -b), ((c0, -1), (c1, -2)))
            c0 -= 2
        #
        c1 -= delta_c0[parity]
        parity = swapParity(parity)
    #
#

In [158]:
for a in range(5, 13, 1):
    for T in directTup_00(a):
        print((T, mrTupValue(T)[0]))

(((5, -2), ((0, -1), (1, -2))), 3)
(((6, -2), ((1, -1), (2, -2))), 6)
(((7, -2), ((2, -1), (3, -2))), 12)
(((7, -2), ((0, -1), (3, -2))), 13)
(((8, -2), ((3, -1), (4, -2))), 24)
(((8, -2), ((1, -1), (4, -2))), 26)
(((9, -2), ((4, -1), (5, -2))), 48)
(((9, -2), ((2, -1), (5, -2))), 52)
(((9, -2), ((0, -1), (5, -2))), 53)
(((10, -2), ((5, -1), (6, -2))), 96)
(((10, -2), ((3, -1), (6, -2))), 104)
(((10, -2), ((1, -1), (6, -2))), 106)
(((10, -2), ((0, -1), (2, -2))), 113)
(((11, -2), ((6, -1), (7, -2))), 192)
(((11, -2), ((4, -1), (7, -2))), 208)
(((11, -2), ((2, -1), (7, -2))), 212)
(((11, -2), ((0, -1), (7, -2))), 213)
(((11, -2), ((1, -1), (3, -2))), 226)
(((11, -2), ((0, -1), (1, -2))), 227)
(((12, -2), ((7, -1), (8, -2))), 384)
(((12, -2), ((5, -1), (8, -2))), 416)
(((12, -2), ((3, -1), (8, -2))), 424)
(((12, -2), ((1, -1), (8, -2))), 426)
(((12, -2), ((2, -1), (4, -2))), 452)
(((12, -2), ((0, -1), (4, -2))), 453)
(((12, -2), ((1, -1), (2, -2))), 454)


In [159]:
for i in range(6, 14, 2):
    print(6*2**(i-6))

6
24
96
384


In [160]:
# The double-zero numbers directly:
def direct_00(a):
    """
    This function directly generates all integers of the given generation $a$
    that have two zeros in their label

    This function give the same answer as brute-force methods such as ZeroSumSet approach
    for large generations (27, 32, 41, ...
    """
    collatzNums = []
    
    def swapParity(parity_):
        return parity_ ^ 1
    #
    
    b = 2
    c1 = a-4
    delta_c01 = [1, 2]
    delta_c0 = [4, 2]
    parity = 0
    while c1 >= 1:
        c0 = c1 - delta_c01[parity]
        while c0 >= 0:
            n = (2**a - 3*(2**c0) - (2**c1))//9
            collatzNums.append(n)
            c0 -= 2
        #
        c1 -= delta_c0[parity]
        parity = swapParity(parity)
    #
    return collatzNums
#

## Number of direct_00 solutions for odd generations >= 3 gives A008810
$$
\large{a(n) = ceiling(\frac{n^{2}}{3})}
$$
So:
$$
   \text{solutionCount(direct\_00(a))}  =  ceiling(\frac{(\frac{a-3}{2})^{2}}{3})
$$

I find this formula version of A008810 interesting as it seems to align with the $\pmod{9}$ nature of the zero sum set finding:
$$
9 \cdot a(n) = 4 + 3n^2 - 2 \cdot A099837(n+3)
$$
Where A099837 is the power series expanion terms in powers of $x$ of the equation:
$$
\frac{1 - x^2}{1 + x + x^2}
$$

## Number of direct_00 solutions for even generations >= 6 gives A007980
$$
\large{a(n) = ceiling(\frac{n^{2} + n}{3})}
$$
$$
\text{Expansion of: } \large{\frac{1+x^2}{(1-x)^2(1-x^3)}}
$$
So:
$$   
   \text{solutionCount(direct\_00(a))}  =  \large{ \frac{(\frac{a-4}{2})^{2} + \frac{a-4}{2}}{3} }
$$


In [161]:
def fit_total_soln_odd():    
    x = []
    y = []
    for i in range(5, 40, 2):
        x.append(i)
        y.append(len(direct_00(i)))
    return x, y
fit_total_soln_odd()

([5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39],
 [1, 2, 3, 6, 9, 12, 17, 22, 27, 34, 41, 48, 57, 66, 75, 86, 97, 108])

In [162]:
# Y is A007980
# 
def fit_total_soln_even():    
    x = []
    y = []
    for i in range(6, 40, 2):
        x.append(i)
        y.append(len(direct_00(i)))
    return x, y
fit_total_soln_even()

([6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38],
 [1, 2, 4, 7, 10, 14, 19, 24, 30, 37, 44, 52, 61, 70, 80, 91, 102])

In [163]:
np.array([1, 2, 4, 7, 10, 14, 19, 24, 30, 37, 44, 52, 61, 70, 80, 91, 102]) - \
   np.array([1, 2, 3, 6, 9, 12, 17, 22, 27, 34, 41, 48, 57, 66, 75, 86, 97])



array([0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5])

In [164]:
[ math.ceil((x**2)/3) for x in range(1, 17, 1) ]

[1, 2, 3, 6, 9, 12, 17, 22, 27, 34, 41, 48, 57, 66, 75, 86]

In [165]:
[ math.ceil((((a-3)/2)**2)/3) for a in range(5, 40, 2) ]

[1, 2, 3, 6, 9, 12, 17, 22, 27, 34, 41, 48, 57, 66, 75, 86, 97, 108]

In [166]:
[ math.ceil((x**2 + x)/3) for x in range(1, 18, 1) ]

[1, 2, 4, 7, 10, 14, 19, 24, 30, 37, 44, 52, 61, 70, 80, 91, 102]

In [167]:
[ math.ceil((((a-4)/2)**2 + ((a-4)/2))/3) for a in range(6, 40, 2) ]

[1, 2, 4, 7, 10, 14, 19, 24, 30, 37, 44, 52, 61, 70, 80, 91, 102]

In [168]:
direct_00(10)

[96, 104, 106, 113]

In [169]:
direct_00(11)

[192, 208, 212, 213, 226, 227]

In [170]:
direct_00(12)

[384, 416, 424, 426, 452, 453, 454]

In [171]:
direct_00(13)

[768, 832, 848, 852, 853, 904, 906, 908, 909]

In [172]:
# This is really fast !
len(direct_00(70))

374

In [173]:
def flipflop(c_a, c_z, delta_tup):
    idx = 0
    c = c_a
    while c >= c_z:
        yield c
        c -= delta_tup[idx]
        idx = (idx + 1) % len(delta_tup)
#
list(flipflop(11, 2, (4, 2)))

[11, 7, 5]

In [174]:
list(flipflop(5-1, 1, (4, 2)))

[4]

In [175]:

def exp1(a):
    C_2 = list(flipflop(a-4, 2, (4, 2)))
    delta_1 = (4, 2)
    for c2_i in range(len(C_2)):
        c2 = C_2[c2_i]
        delta_1 = (delta_1[1], delta_1[0])
        C_1 = list(flipflop(c2 - 3 + c2_i, 1, delta_1))
        for c1_i in range(len(C_1)):
            c1 = C_1[c1_i]
            print((c1, c2))
        
    #
#
exp1(15)

(8, 11)
(6, 11)
(2, 11)
(5, 7)
(1, 7)
(4, 5)
(2, 5)


In [176]:
exp1(16)

(9, 12)
(7, 12)
(3, 12)
(1, 12)
(6, 8)
(2, 8)
(5, 6)
(3, 6)
(2, 2)


# Zero Sum Sets

In examing the modulus limits imposed on the zero indexes, we see that as the number of generations increases beyond 6, 24, etc the zero position choices "gain more freedom to create integers".  For instance modulus choices can only repeat when there are two indexes that would allow a repeat.

## Single zero in label case
As we can see above in the direct_0 function, we can very directly generate all single zero values in the lattice.

- $2^a \pmod{3}$ alternates between 1 and 2
- half of all label bits with indexes less than $a-3$ can produce matches to the leading tuple value term
- label bits are always either all even or odd bits less than $a-3$
    - $2^a, \ a \text{ odd}$:  odd label indexes (1, 3, ...) e.g $|'10111'| = 10$
    - $2^a, \ a \text{ even}$: even label indexes (0, 2, ...) e.g. $|'0111'| = 5$

These numbers have the form:

$$
\frac{1}{3}(2^{2i+p} - 2^{2j+p}) 
$$
and are always integers when:
$$
p \in \{0,1\}
$$
$$
2j < (2i+p) - 3
$$
$$
\text{and in our variable convention, } a = 2i+p
$$
All numbers of this form are in the lattice.

E.g.:
$$
|'0111'| = \frac{1}{3}(2^{2i+p} - 2^{2j+p}) \text{ when } i=2; j=0; p=0
$$
$$
|'10111'| = \frac{1}{3}(2^{2i+p} - 2^{2j+p}) \text{ when } i=2; j=0; p=1
$$

Note that in general, the subtractened item associated with the last zero in the label will always have the same denominator as the initial value term of the tuple.  

## Two zeros in label case

The powers of two (mod 9) cycle through the values $[2,4,8, 7,5,1]$ and in (mod 3) terms this is $[2,2,2, 1,1,1]$ so we expect generational behavior pattern differences across 6 generations.

The last zero cannot have an index that is modulo 6 the same as the leading term because this would require the power of two of the first zero to be $0 \pmod{3^d}$ which is impossible.  Similarly, for even generations for the "two zeros in label case" the last zero cannot have an even index.

$$
\frac{1}{9}(2^{2i+p} - 3\cdot2^{2(2-q)j - p} - 2^{2k + p}) 
$$

TODO: Verify above equation, this is a guess ... and figure out relationship between q,j and other params.  q,j is basically an iterator bounded by $2i+p$, $2k + (1-p)$ ... I think $2i+p$, $2k + (1-p)$ have to be at least 5 apart?

## a=5, b=2: ==============
### 3 = 
$ \frac{2^{5}}{3^{2}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{1}}{3^{2}} ) $
## a=6, b=2: ==============
### 6 = 
$ \frac{2^{6}}{3^{2}} - (   \frac{2^{1}}{3^{1}} + \frac{2^{2}}{3^{2}} ) $
## a=7, b=2: ==============
### 13 = 
$ \frac{2^{7}}{3^{2}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{3}}{3^{2}} ) $
### 12 = 
$ \frac{2^{7}}{3^{2}} - (   \frac{2^{2}}{3^{1}} + \frac{2^{3}}{3^{2}} ) $
## a=8, b=2: ==============
### 26 = 
$ \frac{2^{8}}{3^{2}} - (   \frac{2^{1}}{3^{1}} + \frac{2^{4}}{3^{2}} ) $
### 24 = 
$ \frac{2^{8}}{3^{2}} - (   \frac{2^{3}}{3^{1}} + \frac{2^{4}}{3^{2}} ) $
## a=9, b=2: ==============
### 53 = 
$ \frac{2^{9}}{3^{2}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{5}}{3^{2}} ) $
### 52 = 
$ \frac{2^{9}}{3^{2}} - (   \frac{2^{2}}{3^{1}} + \frac{2^{5}}{3^{2}} ) $
### 48 = 
$ \frac{2^{9}}{3^{2}} - (   \frac{2^{4}}{3^{1}} + \frac{2^{5}}{3^{2}} ) $
## a=10, b=2: ==============
### 113 = 
$ \frac{2^{10}}{3^{2}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{2}}{3^{2}} ) $
### 106 = 
$ \frac{2^{10}}{3^{2}} - (   \frac{2^{1}}{3^{1}} + \frac{2^{6}}{3^{2}} ) $
### 104 = 
$ \frac{2^{10}}{3^{2}} - (   \frac{2^{3}}{3^{1}} + \frac{2^{6}}{3^{2}} ) $
### 96 = 
$ \frac{2^{10}}{3^{2}} - (   \frac{2^{5}}{3^{1}} + \frac{2^{6}}{3^{2}} ) $

In [177]:
def generationModulusChoices(a, b):
    """
    Given a,b with the "standard" meanings:
    a: generation, power of 2 of leading term
    b: number of zeros, power of 3 of denominator

    This function generates all possible 0 index positions for an integer result by testing
    each possibility against the modulus math.

    This is actually more brute force than directly computing the Tuple value and checking for
    a denominator of 1.

    The point is to better understand the structural lattice patterns that lead to integers.
    
    """    
    debug_counter = 0
    denom = 3**b
    leading_term = 2**a % denom
    if a <= 3:
        yield None
    # Added this cache partly to help understand what we are iterating over
    cache = {}
    def get_modulus(c, d):
        if d not in cache:
            cache[d] = {}
        if c not in cache[d]:
            mod = denom - ((((3**(d)) * (2**c))) % denom)
            cache[d][c] = mod
        else:
            mod = cache[d][c]
        return mod
    #
            
    values = range(0, a - 3)
    for item in combinations(values, b):
        debug_counter += 1
        if (debug_counter % 100000000) == 0:
            print(f"{debug_counter} combinations processed")
        current = [leading_term]
        zero_idxes = []
        for i in range(len(item)):
            current.append( get_modulus(item[i], b-i-1) )
            zero_idxes.append(item[i])
        if sum(current) % denom == 0:
            yield (current, denom, zero_idxes)

#    
def countGenerationModulusChoices(a, b):
    n = 0
    for tup in generationModulusChoices(a, b):
        n += 1
    return n
#

In [178]:
def generationLabelChoices(a, b):
    """
    Generate all labels (not just integers) in generation $a$ that have $b$ zeros
    """
    values = range(0, a)
    for item in combinations(values, b):
        L = ["1"]*a
        # Each item is a list of indexes of zero locations ... set them to zero
        for idx in item:
            L[idx] = "0"
        yield "".join(L)
#

In [179]:
list(generationLabelChoices(4, 2))

['0011', '0101', '0110', '1001', '1010', '1100']

In [180]:
for tup in generationModulusChoices(12,3):
    print(tup)
# Useful bits:  [0,1,2,3,4,5, ,8] of 12-3=9.
#    Bits that have more than one position:  [0,1,2,3]
# So only two "impossible" bits: [6,7]

([19, 18, 21, 23], 27, [0, 1, 2])
([19, 18, 3, 14], 27, [0, 3, 8])
([19, 9, 15, 11], 27, [1, 2, 4])
([19, 9, 12, 14], 27, [1, 5, 8])
([19, 18, 3, 14], 27, [2, 3, 8])
([19, 9, 12, 14], 27, [3, 5, 8])


In [181]:
for tup in generationModulusChoices(11,2):
    print(tup)

([5, 6, 7], 9, [0, 1])
([5, 6, 7], 9, [0, 7])
([5, 3, 1], 9, [1, 3])
([5, 6, 7], 9, [2, 7])
([5, 6, 7], 9, [4, 7])
([5, 6, 7], 9, [6, 7])


In [182]:
(len(collatzPath(27)), countZeros(collatzPath(27))), math.comb(70-3, 41)

((70, 41), 2703342303382594104)

So this is never going to scratch a fraction of the posibilities:

In [183]:
"""
for tup in generationModulusChoices(len(collatzPath(27)), countZeros(collatzPath(27))):
    print(tup)
"""

'\nfor tup in generationModulusChoices(len(collatzPath(27)), countZeros(collatzPath(27))):\n    print(tup)\n'

In [184]:
# Brute force find all "two zero" integers in generation 6
for prefix in generationLabels(3):
    label = prefix + "111"
    if countZeros(label) == 2:
        T = mrTupFromPath(label)
        val = mrTupValue(T)
        if val[1] == 1:
            print((label, val[0]))

('100111', 6)


In [185]:
((2**6) - 3*(2**1) - (2**2))/9 

6.0

In [186]:
mrTupValue(mrTupFromPath("10111"))

(10, 1)

In [187]:
# Compute modulus for leading terms
rows = []
for b in range(1, 4, 1):
    for a in range(1,20,1):
        rows.append([a, b, (2**a) % (3**b), 3**b])
df = pd.DataFrame(rows, columns=["a", "b", "modulus", "base"])
df
    

,a,b,modulus,base
0,1,1,2,3
1,2,1,1,3
2,3,1,2,3
3,4,1,1,3
4,5,1,2,3
5,6,1,1,3
6,7,1,2,3
7,8,1,1,3
8,9,1,2,3
9,10,1,1,3


# Min Max per generation per $b$

We find above that $6 \cdot 2^{a-6}$ is consistently the minimum value for even generations and $b=2$.  The **direct_00** algorithm essentially uses the tuple of this number to increasingly subtract larger terms from $6 \cdot 2^{a-6}$ to give all $b=2$ numbers in the lattice.

We have already determined:

- $[min, max](b==0)\ \mapsto 2^a$
    - $[1, 11, 111, 1111, ...]$
- $min(b==1)\ \mapsto \ 5 \cdot 2^{a-4}$
    - $[0111, 10111, 110111, 1110111, ...]$ zero always immediately before final $111$
- $max(b==1)\ \mapsto \ 5 \cdot 2^{a-4} + OEIS:A000975(a-4)$
    - $[0111, 10111, 011111, 1011111, 01111111, 101111111]$ zero always left-most legal position alternating $[0,2]$ positions
    - For generation a > 6:
        - $a \pmod{2} \equiv 0$: $mrTupValue(G_1(G_0(mrTupFromValue(2^{a-2})))$
        - $a \pmod{2} \equiv 1$: $mrTupValue(G_0(G_1(mrTupFromValue(2^{a-2})))$
- $min(b==2)\ \mapsto \ 3 \cdot 2^{a-5}$
    - $[00111, 100111, 1100111, 11100111, ...]$ zeros always immediately before final $111$
- $max(b==2)\ \mapsto $
    - $[00111, 100111, 0110111, 10110111, 011110111, 0101111111, 00111111111, 100111111111, ...]$
    - a > 6:
        - $a \pmod{6} \equiv 0$: $mrTupValue(G_1(G_0(G_0(G_1(G_1(G_1(mrTupFromValue(2^{a-6})))))))$
        - $a \pmod{6} \equiv 1$: $mrTupValue(G_0(G_1(G_1(G_0(G_1(G_1(mrTupFromValue(2^{a-6})))))))$
        - $a \pmod{6} \equiv 2$: $mrTupValue(G_0(G_1(G_1(G_1(G_1(G_0(mrTupFromValue(2^{a-6})))))))$
        - $a \pmod{6} \equiv 3$: $mrTupValue(G_0(G_1(G_0(G_1(G_1(G_1(mrTupFromValue(2^{a-6})))))))$
        - $a \pmod{6} \equiv 4$: $mrTupValue(G_0(G_1(G_0(G_1(G_1(G_1(mrTupFromValue(2^{a-6})))))))$
        - $a \pmod{6} \equiv 5$: $mrTupValue(G_0(G_0(G_1(G_1(G_1(G_1(mrTupFromValue(2^{a-6})))))))$
    - ALSO: $ a > 6, a\&1==1$ : $ OEIS:A136792(a-6) - OEIS:A136792(a-5) $
- $min(b==3)\ \mapsto \ 17 \cdot 2^{a-9}$
    - $[010110111,	1010110111,	11010110111,	111010110111,	1111010110111,	11111010110111,	111111010110111,]$
- $max(b==3)\ \mapsto $
    - $[010110111,	0011110111,	00101111111,	000111111111,	1000111111111,	01100111111111,	101100111111111,	0111100111111111,	01011110111111111,	001111110111111111,	0100111111111111111,	00110111111111111111,	100110111111111111111,	0110110111111111111111,	10110110111111111111111,	010101111111111111111111,	0011101111111111111111111,	10011101111111111111111111,	010110111111111111111111111]$
    - a > 9:
        - $a = 0 (mod\  18)$: '001111110' ...
        - $a = 1 (mod\  18)$: '010011111' ...
        - $a = 2 (mod\  18)$: '001101111' ...
        - $a = 3 (mod\  18)$: '100110111' ...
        - $a = 4 (mod\  18)$: '011011011' ...
        - $a = 5 (mod\  18)$: '101101101' ...
        - $a = 6 (mod\  18)$: '010101111' ...
        - $a = 7 (mod\  18)$: '001110111' ...
        - $a = 8 (mod\  18)$: '100111011' ...
        - $a = 9 (mod\  18)$: '010110111' ...
        - $a = 10 (mod\  18)$: '001111011' ...
        - $a = 11 (mod\  18)$: '001011111' ...
        - $a = 12 (mod\  18)$: '000111111' ...
        - $a = 13 (mod\  18)$: '100011111' ...
        - $a = 14 (mod\  18)$: '011001111' ...
        - $a = 15 (mod\  18)$: '101100111' ...
        - $a = 16 (mod\  18)$: '011110011' ...
        - $a = 17 (mod\  18)$: '010111101' ...

- $min(b==4)\ \mapsto \ 11 \cdot 2^{a-10}$

- $min(b==5)\ \mapsto \ 7 \cdot 2^{a-11}$

## It is pretty clear we will be able to identify the prefix formula for the min/max for a given a,b 

The length of the prefix is determined by the number of bits required to represent $2 \cdot 3^b$

What will the $a$ modulus partition be for $b==4$? ... $54$?

## This is convergence for Collatz!

- IF the reverse minimum value is always increasing per generation per $b$, 
- THEN the forward solution must converge




In [188]:
"""
Cache all Collatz Paths
"""
D = {}
def encodePath(label):
    """
    Save a little memory by storing labels as Python mega integers
    """
    return (len(label), int(label, 2))
def decodePath(lnlabel):
    return f"{lnlabel[1]:0{lnlabel[0]}b}"
def getLabel(nn):
    if nn not in D:
        label = collatzPath(nn)
        D[nn] = encodePath(label)
    else:
        label = decodePath(D[nn])
    return label
#
def next_A092893(tup):
    """
    Use the a,b of the previous A092893 found to 
    constrain the next value search.
    """
    a, b, n = tup
    b_ = b+1
    for i in range(4):
        a_ = a+i+1
        # For a given $a, b$ this is the upper bound of the integer search space.  This 
        # number is not small, but is MUCH smaller than $2^a$.  If we have not found
        # anything in this generation when we reach this upper bound, then on to the 
        # next generation ($a$).
        upper = math.floor((2**(a_))/(3**b_))
        if upper & 1 == 0:
            upper -= 1
        for n_ in range(3, upper, 2):
            label = getLabel(n_)
            if countZeros(label) == b_:
                return (a_, b_, n_)            
    return None
#

# ??? Is there any relationship between the A092893 values and the "illegal" modulus values?

The A092893 are related to the minimum value for a given $b$ and choose the minimum value for all related "$b$ zeros" minimum values in larger generations.

## The "illegal moduli":
- The powers of two (mod 3) cycle through the values $[2,1]$
    - "illegal": $[0]$
- The powers of two (mod 9) cycle through the values $[2,4,8, 7,5,1]$
    - "illegal": $[0, 3, 6]$
:

## What about the label bits that cannot contribute to an integer in a given generation?

The A092893 values at least tell us that some (all?) bits ARE legal


In [189]:
def collectMinMaxInt_a_b_vals(a_max, b_max):
    rows = []
    if a_max < 4:
        return None
    def setMinMax(b_max, row, b, val, label):
        min_idx = 2*b
        max_idx = 2*b + 1
        if row[min_idx] is None or val < row[min_idx]:
            row[min_idx] = val
            row[2*b_max + min_idx] = label
        if row[max_idx] is None or val > row[max_idx]:
            row[max_idx] = val
            row[2*b_max + max_idx] = label
    #
    for a in range(4, a_max + 1, 1):
        row = [None]*(4*(b_max))        
        for prefix in generationLabels(a-3):
            label = prefix + "111"
            b = countZeros(prefix)
            if b < b_max:
                T = mrTupFromPath(label)
                val = mrTupValue(T)
                if val[1] == 1:
                        setMinMax(b_max, row, b, val[0], label)
        rows.append([a, b, a&1] + row)
    #
    return rows
#

# ^^^ Previous Notebook Stuff

In [190]:
collatzPath(25)

'0100110010110111'

In [191]:
# Is it really this simple? 

def mrTupAffine(modulus_bits, mrTup):
    base, subractends = mrTup
    a, b = base
    m = modulus_bits    
    L_ = [(t[0]-m, t[1]+(m-1)) for t in subractends if t[1] <= -m]
    b_ = -len(L_)
    return ((a-m, b_), L_)
#
    

In [192]:
T_25 = mrTupFromValue(25)
T_25

((16, -7), [(0, -1), (2, -2), (3, -3), (6, -4), (7, -5), (9, -6), (12, -7)])

In [193]:
25 % 8

1

$1 \pmod{8} \mapsto 010 \mapsto [(0, -1), (2, -2)] \mapsto \frac{7}{9} $

In [194]:
T_29 = mrTupAffine(3, T_25)
T_29, mrTupValue(T_29)

(((13, -5), [(0, -1), (3, -2), (4, -3), (6, -4), (9, -5)]), (29, 1))

In [195]:
29 % 8

5

$5 \pmod{8} \mapsto 011 \mapsto [(0, -1)] \mapsto \frac{1}{3} $

### Goal: 010 011 001 011 0111
#### 010 011

In [196]:
T_34 = mrTupAffine(3, T_29)
T_34, mrTupValue(T_34)

(((10, -3), [(1, -1), (3, -2), (6, -3)]), (34, 1))

In [197]:
34 % 2

0

### Goal: 010 011 001 011 0111
####      010 011 ?

In [198]:
# The extra divide by 2 messes up the pattern ...
34 // 2

17

In [199]:
T_17 = G_rev(T_34)
T_17, mrTupValue(T_17)

(((9, -3), [(0, -1), (2, -2), (5, -3)]), (17, 1))

In [200]:
17 % 8

1

$1 \pmod{8} \mapsto 010 \mapsto [(0, -1), (2, -2)] \mapsto \frac{7}{9} $

### Goal: 010 011 001 011 0111
####      010 011 ? 010

In [201]:
T_20 = mrTupAffine(3, T_17)
T_20, mrTupValue(T_20)

(((6, -1), [(2, -1)]), (20, 1))

In [202]:
20 % 4

0

In [203]:
### Goal: 010 011 001 011 0111
####      010 011 ? 010 ?? 

In [204]:
T_5 = G_rev(G_rev(T_20))
T_5, mrTupValue(T_5)

(((4, -1), [(0, -1)]), (5, 1))

In [205]:
5 % 8

5

$5 \pmod{8} \mapsto 011 \mapsto [(0, -1)] \mapsto \frac{1}{3}$
### Goal: 010 011 0 010 11 0111
####      010 011 ? 010 ?? 011 _2

In [206]:
(26 % 8, 26//2), (13 % 8, T_011_to_T_111(13) // 8), (5 % 8, T_011_to_T_111(5)//8)

((2, 13), (5, 5), (5, 2))

In [207]:
def pathUsingAffine(n):
    label_parts = []

    """
    This is effectively just a restatement of the Collatz Conjecture with 8 options instead of 2, 
    with each option compositing 3 Shortcut Collatz steps.
    """
    
    while n not in [1,2,4]:
        mod8 = n % 8
        if mod8 == 0:
            label_parts.append("111")
            n //=8
        elif mod8 == 4:
            label_parts.append("11")
            n //= 4
        elif mod8 == 2 or mod8 == 6:
            label_parts.append("1")
            n //= 2
        elif mod8 == 1:
            label_parts.append("010")
            n = T_010_to_T_111(n) // 8
        elif mod8 == 3:
            label_parts.append("001")
            n = T_001_to_T_111(n) // 8
        elif mod8 == 5:
            label_parts.append("011")
            n = T_011_to_T_111(n) // 8
        elif mod8 == 7:
            label_parts.append("000")
            n = T_000_to_T_111(n) // 8
    if n == 2:
        label_parts.append("1")
    elif n == 4:
        label_parts.append("11")
            
    return "".join(label_parts)
#
        

# Regular Collatz operation

So the Collatz 3n+1 operation is just a single Affine operation that changes the leading label $0$ of an odd number to $1$.

So 3n+1 REMOVES the first subtractend from the mixed radix form.

The 3n+1 operation DOES NOT move further from $1$ in this way of looking at things. It is an "up" operation, not a left/right operation. It shifts up to the even part of the lattice which is entirely above the odd part of the lattice.

The distance from 1 is then always the number of divide-by-2 operations required

In [208]:
collatzPath(25), collatzPath(76)

('0100110010110111', '1100110010110111')

In [209]:
pathUsingAffine(25), collatzPath(25)

('0100110010110111', '0100110010110111')

In [210]:
for n in range(9, 100, 1):
    label = collatzPath(n)
    label_ = pathUsingAffine(n)
    if label != label_:
        print((n, label, label_))

In [211]:
T_2 = mrTupAffine(3, T_5)
T_2, mrTupValue(T_2)

(((1, 0), []), (2, 1))

In [212]:
def sumSubtractends(L):
    val = (0, 1)
    accum = 0
    if len(L) == 1:
        val = (2**L[0][0], 3**(-L[0][1]))
    elif len(L) > 0:
        for tup in L:
            accum += ( (3**(tup[1] - L[-1][1]))*(2**tup[0]) )
        # print(accum)
        # print(-L[-1][1])
        # print(L[0][1]-L[-1][1]+1)
        f = Fraction(accum, (3**(-L[-1][1])))
        val = (f.numerator, f.denominator)

    return val
#

# What do these lattices look like?

In [266]:
src = []
dst = []

# What are the labels of our two 2n+1 lattices?
for prefix in shortenedGenerationLabels(9):
    if prefix[-1] == "0":
        label = prefix[0:-1] + "111"
        T = mrTupFromPath(label)
        val = mrTupValue(T)
        if val[1] == 1:
            T_ = mr2Nplus_1(T)
            if T_ is None:
                dst.append((len(label), label[::-1], val[0]))
            else:
                src.append((len(label), label[::-1], val[0]))
    #
    label = prefix + "111"
    T = mrTupFromPath(label)
    val = mrTupValue(T)
    if val[1] == 1:
        T_ = mr2Nplus_1(T)
        if T_ is None:
            dst.append((len(label), label[::-1], val[0]))
        else:
            src.append((len(label), label[::-1], val[0]))


In [267]:
sorted(src)

[(5, '11101', 10),
 (6, '111001', 6),
 (7, '1111101', 42),
 (8, '11101101', 26),
 (9, '111011010', 17),
 (9, '111111101', 170),
 (10, '1110110100', 11),
 (10, '1110110101', 34),
 (10, '1110111101', 106),
 (10, '1111111010', 113),
 (11, '11101101000', 7),
 (11, '11101101001', 22),
 (11, '11101111001', 70),
 (11, '11111110100', 75),
 (11, '11111110101', 226),
 (11, '11111111101', 682),
 (12, '111011010001', 14),
 (12, '111011011101', 138),
 (12, '111011110001', 46),
 (12, '111011111101', 426),
 (12, '111111101001', 150),
 (12, '111111111001', 454)]

In [268]:
sorted(dst)

[(3, '111', 8),
 (4, '1110', 5),
 (4, '1111', 16),
 (5, '11100', 3),
 (5, '11111', 32),
 (6, '111011', 20),
 (6, '111110', 21),
 (6, '111111', 64),
 (7, '1110011', 12),
 (7, '1110110', 13),
 (7, '1110111', 40),
 (7, '1111111', 128),
 (8, '11100111', 24),
 (8, '11101111', 80),
 (8, '11111011', 84),
 (8, '11111110', 85),
 (8, '11111111', 256),
 (9, '111001111', 48),
 (9, '111011011', 52),
 (9, '111011110', 53),
 (9, '111011111', 160),
 (9, '111110111', 168),
 (9, '111111111', 512),
 (10, '1110011111', 96),
 (10, '1110110111', 104),
 (10, '1110111100', 35),
 (10, '1110111111', 320),
 (10, '1111101111', 336),
 (10, '1111111011', 340),
 (10, '1111111110', 341),
 (10, '1111111111', 1024),
 (11, '11100111111', 192),
 (11, '11101101011', 68),
 (11, '11101101110', 69),
 (11, '11101101111', 208),
 (11, '11101111000', 23),
 (11, '11101111011', 212),
 (11, '11101111110', 213),
 (11, '11101111111', 640),
 (11, '11111011111', 672),
 (11, '11111110111', 680),
 (11, '11111111100', 227),
 (11, '1111111

In [ ]:
# ('11101101', 26) 
# => 
# ('011110111', 53)

# (mod 2) possibilities for 3n+1

### Always:  
- 0(mod 2) -|3n+1|-> 1
- 1(mod 2) -|3n+1|-> 0


# (mod 8) possibilities for 3n+1

### Always:  
- 0(mod 8) -|3n+1|-> 1
- 1(mod 8) -|3n+1|-> 4
- 2(mod 8) -|3n+1|-> 7
- 3(mod 8) -|3n+1|-> 2
- 4(mod 8) -|3n+1|-> 5
- 5(mod 8) -|3n+1|-> 0
- 6(mod 8) -|3n+1|-> 3
- 7(mod 8) -|3n+1|-> 6

### And of course the way we create an odd number from an even number is "divide by 2"





# Odd to Even Affine versus Even to Odd

## Odd to Even

- $3n+1$ : $G_{1}(G^{-1}(T))$ -- replace the first zero with a 1

## Even to Odd

- n(mod 3):
    - 2: $G_0(T)$
    - 1: $G_0(G_1(T))$
    - 0: No general label manipulation solution! -- Must divide by 2 until we have an odd number to get an odd number.

#### The Modular Obstacle for even 0(mod 3)

In the standard Collatz graph, no power of 3 (other than 1) can be the result of a $3n+1$ operation.If $3n+1 = 3k$, then $1 = 3(k-n)$, which is impossible.Consequently, in the inverse lattice, a node that is $0 \pmod 3$ cannot have a "0" child. It can only have "1" children ($2n, 4n, 8n \dots$).


# Adjacent zeros integers

Because "00" makes it "harder" to be an integer lets look at some integers with adjacent zeros.

My intutition is they need to be "balanced" later

In [339]:
for prefix in shortenedGenerationLabels(9):
    if prefix.find("00") != -1:
        label = prefix + "111"
        T = mrTupFromPath(label)
        val = mrTupValue(T)
        if val[1] == 1:
            print((label, val))


('111111100111', (384, 1))
('1111100111', (96, 1))
('11100111', (24, 1))
('110011110111', (140, 1))
('110010110111', (44, 1))
('100111111111', (454, 1))
('100101111111', (150, 1))
('100011110111', (46, 1))
('100010110111', (14, 1))
('100111', (6, 1))
('0011110111', (35, 1))
('0010110111', (11, 1))
('000111111111', (151, 1))
('000011110111', (15, 1))
